## Table of Contents
#### Preliminaries
- [adding abilities to strategies_dict](#adding_abilities)
- [Cleaning up the target feature (formats)](#cleaning_formats)
- [pokemon_types_summary](#pokemon_types_summary)

#### Feature Engineering: Shrinking Moves and Abilities!
- [Handling duplicate moves](#dup_moves)
- [Explaining the cindex feature to shrink moves and abilities](#explain_cindex)
- [Handling duplicate abilities](#dup_abilities)
- [Throwing the data into PCA](#pca_throw)
- [Creating the moves_cindex dataframe](#moves_cindex_df)
- [Creating the abilities_cindex dataframe](#abilities_cindex_df)
- [Investigating Roles for Features](#role_features)
    - [Utility: Entry Hazards](#utility_entry)
    - [Utility: Hazard Control](#hazard_control)
    - [Utility: Clerics and Wish](#utility_clerics)
    - [Utility: Other](#utility_other)
    - [Offensive: Setup](#offense_setup)
    - [Offensive: Priority](#offense_priority)
    - [Defensive: HP Drain and Recovery](#defense_drain)
    - [Weather](#weather)
    - [Indiscriminate Moves](#indiscriminate)
- [Investigating moves_cindex](#moves_cindex_invest)
- [Aggregate Feature: Physical Cutoff](#physical_cutoff)
- [Aggregate Feature: Physical Coverage](#physical_coverage)
- [Aggregate Feature: Special Cutoff](#special_cutoff)
- [Aggregate Feature: Special Coverage](#special_coverage)
- [Aggregate Feature: Miscellaneous Status](#misc_status)
- [Unique Powerful Moves Feature](#unique_powerful)
- [Investigating abilities_cindex](#abilities_invest)
- [Aggregate Feature: Ability Cutoff](#ability_cutoff)
- [Other Ability Features](#other_ability)

#### Feature Selection and Visualizations
- [Removing Obviously Unnecessary Features](#removing_unnecessary)
- [Principal Component Analysis](#pca)
- [Preparing the Data for OOTB Modeling](#data_prep)
- [Logistic Regression with L1 norm](#logistic_l1)
- [Random Forest Feature Importances](#random_forest)
- [Hierarchical Clustering](#hierarchical_clustering)
- [k-means clustering](#k_means)
- [DBSCAN: Density-Based Clustering](#DBSCAN)
- [Mean shift clustering](#MeanShift)
- [T-distributed Stochastic Neighbor Embedding](#tsne)
- [Heatmap of Feature Correlations](#heatmap)
- [Plotting features against the Target](#target_plots)


- [What To Test During Pre-Processing and Modeling](#what_test)

### imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import json
import pandas_profiling
import requests
from bs4 import BeautifulSoup
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import precision_recall_fscore_support, log_loss, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.cluster import KMeans, DBSCAN, MeanShift
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [ ]:
pokemon_df = pd.read_csv('./data/pokemon_df.csv', index_col="name")
pokemon_types_df = pd.read_csv('./data/pokemon_types_df.csv', index_col="name")
pokemon_abilities_df = pd.read_csv('./data/pokemon_abilities_df.csv', index_col="name")
abilities_df = pd.read_csv('./data/abilities_df.csv', index_col="name")
moves_df = pd.read_csv('./data/moves_df.csv', index_col="name")
abilities_removed_df = pd.read_csv('./data/abilities_removed_df.csv')
abilities_removed_df = abilities_removed_df.iloc[:, 0]
abilities_removed_df.name = 'name'
pokemon_learnsets_df = pd.read_csv('./data/pokemon_learnsets_df.csv', index_col='name')

## Preliminaries

<a id='adding_abilities'></a>
### adding abilities to strategies_dict

When I made strategies_dict during data wrangling, I made it to hold the competitive movesets of each pokemon with the name of the role they played, so that it would be easier to sort and extract insight from these competitive movesets and roles later.  However, I made a crucial oversight: pokemon also have multiple abilities to chose from, and only some are used competitively.  Therefore, I should also put those abilities that pokemon choose over their other abilities in strategies_dict as well.  We can adapt our code from data wrangling to make this much easier (most of this was in data wrangling already where the heavy duty experimentation to clean up the strings took place):

In [ ]:
with open("smogonpokemondata2021/smogonpokemondata2021/scraped_data/PokemonData2021.csv", encoding="utf8") as infile:
    strategydex_df = pd.read_csv(infile)

strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('<p>.+?<\/p>', '', regex=True)
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('<section>.+?<\/section>', '', regex=True)
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('<h1>.+?<\/h1>', '', regex=True)
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('\\\\n', '', regex=True)
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace("\'", '\"')
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('King\"s', "King\'s")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Maki\"s', "Maki\'s")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Land\"s', "Land\'s")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Dragon\"s', "Dragon\'s")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Sirfetch\"d', "Sirfetch\'d")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Farfetch\"d', "Farfetch\'d")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Nature\"s', "Nature\'s")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Forest\"s', "Forest\'s")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('drampa\"s', "drampa\'s")
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('False', '\"\"')
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('True', '\"\"')
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('None', '\"\"')
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('\"\"\"\s', '\"', regex=True)
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('<ul>.+?<\/ul', '', regex=True)
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Swipe', 'False Swipe') #newline
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].str.replace('Surrender', 'False Surrender') #newline
strategydex_df['pokemonData'] = strategydex_df['pokemonData'].apply(json.loads)

strategydex_df.head()

There are a lot of formats we don't have any intention to consider (CAP, Monotype, National Dex, VGC20, etc.) with strategies that we scraped, so we should limit the formats of abilities and moves we pick up to only those within acceptable formats:

In [ ]:
acceptable_formats = pokemon_df['formats'].unique().tolist()
acceptable_formats.append('ZU')
acceptable_formats

In [ ]:
strategies_dict = {}
strategydexcopy = strategydex_df.copy()

for pokemon in pokemon_df.index:
    pokemon_access = pokemon.replace("'", "").replace(" ", "-").replace(".", "").replace("-10%", "").replace(":", "")
    dex_entry = strategydexcopy.loc[strategydexcopy['pokemonName'] == pokemon_access.lower(), 'pokemonData'].item()
    format_list = []
    for competitive_format in dex_entry['strategies']:
        format_dict = {}
        current_format = competitive_format['format']
        if current_format in acceptable_formats:
            moveset_list = []
            for moveset in competitive_format['movesets']:
                moveset_dict = {}
                moveset_name = moveset['name']
                move_list = []
                for move in moveset['moveslots']:
                    for entry in move:
                        move_list.append(entry['move'])
                moveset_dict[moveset_name] = move_list
                moveset_dict["abilities"] = moveset['abilities']
                moveset_list.append(moveset_dict)
            format_dict[current_format] = moveset_list
            format_list.append(format_dict)
    strategies_dict[pokemon] = format_list

By simply making minor additions and tweaks to our previous code, strategies_dict has been built again with more information than before!

<a id='cleaning_formats'></a>
### Cleaning up the target feature (formats)

In [ ]:
pokemon_df['formats'].unique()

Our formats column, which is what we are trying to predict, is still messy because for our project, some of these headings mean the same thing.  LC (Little Cup) and NFE (Not Fully Evolved) will simply be moved to the least competitive tier of pokemon, ZU, which is also not even technically present (it really is, but under the name 'Untiered'). Finally, all those with 'BL' in the name will be moved to one tier higher, since they are on the banlist for their current tier and thus the lowest tier they can actually be used in is one higher.

In [ ]:
pokemon_df.loc[pokemon_df['formats'] == 'LC', 'formats'] = 'ZU'
pokemon_df.loc[pokemon_df['formats'] == 'NFE', 'formats'] = 'ZU'
pokemon_df.loc[pokemon_df['formats'] == 'Untiered', 'formats'] = 'ZU'
pokemon_df.loc[pokemon_df['formats'] == 'PUBL', 'formats'] = 'NU'
pokemon_df.loc[pokemon_df['formats'] == 'NUBL', 'formats'] = 'RU'
pokemon_df.loc[pokemon_df['formats'] == 'RUBL', 'formats'] = 'UU'
pokemon_df.loc[pokemon_df['formats'] == 'UUBL', 'formats'] = 'OU'
pokemon_df['formats'].unique()

Fixed.  Now let's examine the distribution of the actual tiers (classes) we'll be working with:

In [ ]:
pokemon_df['formats'].value_counts()

This is a slight problem: some of my classes have under 40 examples, which can be very problematic to train a classifier on, especially with such an unbalanced class like ZU.  If I split into training and testing data (which I surely will), then those classes will have under 30 samples which usually is not even enough to approximate a normal distribution (30 samples).  We can do two things about this:

- (Two-Stage Model) One is that we'll probably want to do a multi-stage model, where we separate ZU from the rest of the classes, and then have a second model to classify the pokemon left standing (i.e. the one's that aren't ZU a.k.a. uncompetitive) into one of the 6 higher competitive tiers.

- (Class Padding) However, even that will be challenging when the other classes are so small.  One nice thing about this data is that the target variable is a bit arbitrary.  That doesn't mean it can just be anything, otherwise this project would be pointless. That said, because it's decided by humans (albeit experts), and because pokemon ARE able to be used in higher tiers (but not lower tiers), formats could arguably be measured in another way.  Instead of taking the direct format rank of the pokemon, we can rank by the rank of their HIGHEST competitive strategy (since pokemon used in a higher tier, even if it's not their main tier, are arguably better than pokemon not used in that higher tier who nonetheless share the same main tier).

Since class padding is another way to legitimately organize the data, we can have two versions of the data: one with padded tiers, and another with our original tiers, and we can test model performance on both and see which does better.  We still need to figure out precisely the best way to pad the classes though.

First of all, I'm going to need to have a list of formats ordered by how high the format is, so if the pokemon has a moveset for such a format, then I don't have to check the lower formats (and also I need a database to hold this information about tier changes):

In [ ]:
ranked_formats = ['Uber', 'OU', 'UU', 'RU', 'NU', 'PU', 'ZU']

tierchange_df = pd.DataFrame(index=pokemon_df.index, columns=['Tier', 'Change'])
tierchange_df['Tier'] = pokemon_df['formats']
tierchange_df.head()

When I tried different methods of class padding in my experimentation notebook, I found that only choosing the highest competitive strategy did balance the classes somewhat, especially taking a lot of pokemon out of ZU, but it didn't quite balance them all to the threshold of 40 per class that I was looking for.  I experimented with various methods for restricting how far a pokemon could move up, which is arguably a sensible thing to do anyways (even if a ZU pokemon, the lowest tier, has an Uber strategy, the highest tier, it may be misleading to really consider them an "Uber", since they may not be successful in most higher tiers and just have a narrow niche in Ubers). The experiments we did can be found in the section "tierchange_df experiments" in the file [EDA_experiments](./EDA_experiments.ipynb#tierchange_df_experiments), but for our purposes I will just say what worked best:

Pokemon in the two lowest tiers (PU and ZU , the 6th and 7th) could rise as high as NU (the 5th tier out of 7), but no higher.  Pokemon in NU and RU (5th and 4th tiers) could rise as high as UU (the 3rd tier out of 7), but no higher.  And finally, only pokemon in UU and OU could rise as high as Ubers (the highest tier), and of course Ubers can't possibly rise anymore.  This lead to the following results:

In [ ]:
for pokemon, formatlist in strategies_dict.items():
    possible_tiers = []
    for formatdict in formatlist:
        for key in formatdict.keys():
            possible_tiers.append(key)
    best_tier = tierchange_df.loc[pokemon, 'Tier']
    if best_tier == 'OU':
        if 'Uber' in possible_tiers:
            best_tier = 'Uber'
    elif best_tier == 'UU':
        if 'Uber' in possible_tiers:
            best_tier = 'Uber'
        elif 'OU' in possible_tiers:
            best_tier = 'OU'
    elif best_tier == 'RU':
        if 'UU' in possible_tiers:
            best_tier = 'UU'
    elif best_tier == 'NU':
        if 'UU' in possible_tiers:
            best_tier = 'UU'
        elif 'RU' in possible_tiers:
            best_tier = 'RU'
    elif best_tier == 'PU':
        if 'NU' in possible_tiers:
            best_tier = 'NU'
    elif best_tier == 'ZU':
        if 'NU' in possible_tiers:
            best_tier = 'NU'
        elif 'PU' in possible_tiers:
            best_tier = 'PU'
    tierchange_df.loc[pokemon, 'Change'] = best_tier

tierchange_df['Change'].value_counts()

This gives us the class sizes of at least 40 that we were looking for!

However, we need to inspect it to see if we are happy with the results.  Let's look at the reasonable sized higher tiers (NU through Ubers, since looking through hundreds of pokemon in ZU and PU is both infeasible and unnecessary since they are all so low anyways) to see what tier each pokemon in it originated from if it wasn't originally in the tier, and check it against the smogon viability rankings for the tier.  If the pokemon isn't in the viability rankings for the higher tier, then they don't deserve to move up even if they DO have a strategy, since that strategy is considered unviable by experts:

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None):
    print(tierchange_df.loc[(tierchange_df['Change'] == 'Uber') & (tierchange_df['Tier'] != 'Uber')].sort_values(by=['Tier']))

According to the Ubers and OU viability rankings (compiled by appointed pokemon experts) at the point of time of writing this (keep in mind it may change later especially with major updates, because data in the real world changes unfortunately):
- https://www.smogon.com/forums/threads/crown-tundra-ss-ubers-viability-rankings.3675194/
- https://www.smogon.com/forums/threads/ss-ou-dlc2-viability-ranking-thread-see-post-1517-on-page-61.3674058/

**Changed Ubers that are too high:**
- Down to UU: Tentacruel, Zygarde-10%
- Down to OU: Tapu Bulu

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None):
    print(tierchange_df.loc[(tierchange_df['Change'] == 'OU') & (tierchange_df['Tier'] != 'OU')].sort_values(by=['Tier']))

All of these are acceptable changes according to OU viability rankings which were already cited

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None):
    print(tierchange_df.loc[(tierchange_df['Change'] == 'UU') & (tierchange_df['Tier'] != 'UU')].sort_values(by=['Tier']))

With the same caveats as before, according to UU and RU viability rankings:
- https://www.smogon.com/forums/threads/ss-uu-crown-tundra-viability-ranking-thread.3674793/
- https://www.smogon.com/forums/threads/ss-ru-viability-rankings-crown-tundra.3676023/

**Changed UU that are too high:**
- Down to RU: Slurpuff

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None):
    print(tierchange_df.loc[(tierchange_df['Change'] == 'RU') & (tierchange_df['Tier'] != 'RU')].sort_values(by=['Tier']))

According to RU viability rankings already cited above:

**Changed RU that are too high:**
- Down to NU: Inteleon, Mudsdale, Drapion, Zoroark, Glastrier, Omastar

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None):
    print(tierchange_df.loc[(tierchange_df['Change'] == 'NU') & (tierchange_df['Tier'] != 'NU')].sort_values(by=['Tier']))

With the same caveats as before, according to UU and RU viability rankings:

**Changed NU that are too high:**
- Down to PU: Silvally-Fairy, Hitmonlee, Charizard, Sneasel, Rotom, Stunfisk-Galar, Centiskorch

**Also**:
- Silvally-Ground gets to move up to NU, since it has a presence in the viability rankings there

**For PU, I'm not going to bother looking at this because there are too many and if you have a PU strategy then you are almost certainly better than pure ZU pokemon which have no ranked strategy at all**

Now we just need to make the changes we observed needed to be made, and place them as the new formats in pokemon_df (I will still have access to the old formats, if I desire them, by simply loading the data as we did in the beginning of this project):

In [ ]:
tierchange_df.loc['Tentacruel', 'Change'] = 'UU'
tierchange_df.loc['Zygarde-10%', 'Change'] = 'UU'
tierchange_df.loc['Tapu Bulu', 'Change'] = 'OU'
tierchange_df.loc['Slurpuff', 'Change'] = 'RU'
tierchange_df.loc['Inteleon', 'Change'] = 'NU'
tierchange_df.loc['Mudsdale', 'Change'] = 'NU'
tierchange_df.loc['Drapion', 'Change'] = 'NU'
tierchange_df.loc['Zoroark', 'Change'] = 'NU'
tierchange_df.loc['Glastrier', 'Change'] = 'NU'
tierchange_df.loc['Omastar', 'Change'] = 'NU'
tierchange_df.loc['Silvally-Ground', 'Change'] = 'NU'
tierchange_df.loc['Silvally-Fairy', 'Change'] = 'PU'
tierchange_df.loc['Hitmonlee', 'Change'] = 'PU'
tierchange_df.loc['Charizard', 'Change'] = 'PU'
tierchange_df.loc['Sneasel', 'Change'] = 'PU'
tierchange_df.loc['Rotom', 'Change'] = 'PU'
tierchange_df.loc['Stunfisk-Galar', 'Change'] = 'PU'
tierchange_df.loc['Centiskorch', 'Change'] = 'PU'

pokemon_df['formats'] = tierchange_df['Change']
pokemon_df['formats'].value_counts()

And our final counts are still acceptable even though the data is more robust than ever, so we can go to the next step:

## Feature Engineering

Feature engineering is by far the most important part of this EDA, because our data is extremely wide (well over 700 columns) for a dataset that only has 738 rows. If kept as is, our model won't have any degrees of freedom.  A general rule of thumb is that there should be about 10 times fewer columns than rows (at a maximum), and for multi-class classication (which is what we're doing), it's really expected that there are even fewer columns than that.  That means we need to shrink nearly 1000 columns into fewer than 70, ideally a lot fewer than 70, while still preserving the essential information enough that the model can perform well. That's no trivial challenge.

<a id='pokemon_types_summary'></a>
### pokemon_types_summary

Let's begin with types, because it's the easiest of my 4 categories to work with (besides stats, which only has 6 very important features so nothing significant needs to be done with it).  The other 2 are abilities and moves, and they will be a much more severe challenge.  Types have 17 columns, representing the type of each pokemon, which should be more manageable to reduce in dimensionality:

- Instead of type columns, let's transform type data into type charts which tell how strong pokemon overall are against each type. I will also aggregate the weaknesses and resistances into a single number (resistance index being amount of resistance minus amount of weakness, so that the higher the resistance index is, the better the pokemon's type performs defensively)
- I should also include a column that would give some idea of the pokemon’s stab moves (this just indicates whether they have one or two types, because two typed pokemon have a slight offensive advantage due to having more STAB moves).  I doubt this will be as important as the resistance index, but it's easy to do so I might as well.

I will aggregate type information into 5 numbers at first (not including the STAB column):
- Number of standard weaknesses (2x)
- Number of severe weaknesses (4x)
- Number of standard resistances (0.5x)
- Number of strong resistances (0.25x)
- Number of total resistances (0x)

The best strategy is probably using requests and beautiful soup to get the type resistances I need from a page like this that lists each type relationship as a number: https://bulbapedia.bulbagarden.net/wiki/Type

In [ ]:
r = requests.get('https://bulbapedia.bulbagarden.net/wiki/Type')
html_doc = r.text
soup = BeautifulSoup(html_doc)
type_table = soup.find_all('table')[1]
type_rows = type_table.find_all('tr')
type_list = []
for th in type_rows[1].find_all('th'):
    type_prep = th.a.get('href')
    type_list.append(type_prep.replace('/wiki/', '').replace('_(type)', ''))

type_list

So here is the type matchups table from Bulbapedia (generation 8 of pokemon) rendered as a dataframe:

In [ ]:
type_matchups = pd.DataFrame(index=type_list, columns=type_list)

for i1, tr in enumerate(type_rows[2:20]):
    for i2, td in enumerate(tr.find_all('td')):
        type_matchups.iloc[i1, i2] = float(td.contents[0][0].replace('½', '0.5'))
        
type_matchups.index.rename('Attacking', inplace=True)
type_matchups.columns.rename('Defending', inplace=True)
        
type_matchups

From this, we can apply each row of type_matchups to pokemon_types_df, which lists the types each pokemon has, multiplying each row of type_matchups with each pokemon that has that type (starting from a base value of 1 for the standard resistance against each type), which will give the overall effectiveness of each pokemon against each type.  And from that, we can make the summary variables to contain the type information in a fraction of the previous number of columns:

In [ ]:
pokemon_types_matchups = pd.DataFrame(1, index=pokemon_types_df.index, columns=type_matchups.columns.rename(''))
for column in pokemon_types_df.columns:
    type_matchup = type_matchups[column]
    type_filter = pokemon_types_df[column] == 1
    pokemon_types_matchups[type_filter] = pokemon_types_matchups[type_filter] * type_matchup

pokemon_types_matchups.head()

That worked perfectly!  Now I can go through all of these columns, and easily produce the five summary variable features:
- Number of standard weaknesses (2x), Weaknesses
- Number of severe weaknesses (4x), Strong Weaknesses
- Number of standard resistances (0.5x), Resists
- Number of strong resistances (0.25x), Strong Resists
- Number of total resistances (0x), Immune
- Plus another column for number of STAB moves, essentially number of types the pokemon has which is very easy to figure out

If we want to condense even further, we can create a single resistance index for each pokemon which would be something like the following:
- Resistance Index = Resists + 2(Strong Resists) + 4(Immune) - Weaknesses - 2(Strong Weaknesses)

And with something like that, we can reduce how powerful a pokemon's type is to a single number (imperfectly of course).

In [ ]:
pokemon_types_summary = pd.DataFrame(index=pokemon_types_matchups.index, columns=['2', '4', '0.5', '0.25', '0'])
for column in pokemon_types_summary.columns:
    pokemon_types_summary[column] = (pokemon_types_matchups == float(column)).sum(axis=1)
    
pokemon_types_summary.columns = ['Weaknesses', 'Strong Weaknesses', 'Resists', 'Strong Resists', 'Immune']
pokemon_types_summary['STAB'] = pokemon_types_df.sum(axis=1)
pokemon_types_summary['Resistance Index'] = pokemon_types_summary['Resists'] + 2*pokemon_types_summary['Strong Resists'] + 4*pokemon_types_summary['Immune'] - pokemon_types_summary['Weaknesses'] - 2*pokemon_types_summary['Strong Weaknesses']

pokemon_types_summary.head()

## Feature Engineering: Shrinking Moves and Abilities!

In [ ]:
pokemon_learnsets_df.shape

<a id="dup_moves"></a>
### Handling duplicate moves

In our dataset we currently have 644 moves! This is the feature that requires the most dimensionality reduction because it is eliminating all degrees of freedom in our model, but first there is something misleading: some of these moves are exactly the same!  That is to say, the only thing different about the moves is their name, but their effects are identical. This is most often the case for status moves, since physical and special attacking moves have different types and thus can have quite different effects when used on or by different pokemon, even if they have the same power, accuracy, secondary effect (e.g. Flinch, % chance for a status condition, etc.), etc.  Type can occasionally be relevant for status moves, but isn't relevant anywhere near as often as it is for physical and special moves so we can ignore it. Grouping moves that are the same in nearly all relevant respects together is important not to mislead any model we build, since if kept separate, not only are we giving the model unnecessary information, we're having it count the same move (or virtually the same move) differently, which makes no sense.

It was easy to use this research to determine which moves are identical (it's not everything on that list, just a significant portion of the status moves): https://bulbapedia.bulbagarden.net/wiki/Move_variations

- Acid Armor, Iron Defense
- Rock Polish, Agility
- Charm, Feather Dance
- Smokescreen, Sand Attack
- Heal Pulse, Floral Healing
- Leer, Tail Whip
- Lock-On, Mind Reader
- Mean Look, Block
- Healing Wish, Lunar Dance
- Moonlight, Morning Sun, Synthesis
- Recover, Soft-Boiled, Milk Drink, Slack Off
- Trick, Switcheroo

In [ ]:
pokemon_learnsets_df['Acid Armor'] = pokemon_learnsets_df['Acid Armor'] + pokemon_learnsets_df['Iron Defense']
pokemon_learnsets_df.drop('Iron Defense', inplace=True, axis=1)
pokemon_learnsets_df['Agility'] = pokemon_learnsets_df['Agility'] + pokemon_learnsets_df['Rock Polish']
pokemon_learnsets_df.drop('Rock Polish', inplace=True, axis=1)
pokemon_learnsets_df['Charm'] = pokemon_learnsets_df['Charm'] + pokemon_learnsets_df['Feather Dance']
pokemon_learnsets_df.drop('Feather Dance', inplace=True, axis=1)
pokemon_learnsets_df['Smokescreen'] = pokemon_learnsets_df['Smokescreen'] + pokemon_learnsets_df['Sand Attack']
pokemon_learnsets_df.drop('Sand Attack', inplace=True, axis=1)
pokemon_learnsets_df['Heal Pulse'] = pokemon_learnsets_df['Heal Pulse'] + pokemon_learnsets_df['Floral Healing']
pokemon_learnsets_df.drop('Floral Healing', inplace=True, axis=1)
pokemon_learnsets_df['Leer'] = pokemon_learnsets_df['Leer'] + pokemon_learnsets_df['Tail Whip']
pokemon_learnsets_df.drop('Tail Whip', inplace=True, axis=1)
pokemon_learnsets_df['Lock-On'] = pokemon_learnsets_df['Lock-On'] + pokemon_learnsets_df['Mind Reader']
pokemon_learnsets_df.drop('Mind Reader', inplace=True, axis=1)
pokemon_learnsets_df['Block'] = pokemon_learnsets_df['Block'] + pokemon_learnsets_df['Mean Look']
pokemon_learnsets_df.drop('Mean Look', inplace=True, axis=1)
pokemon_learnsets_df['Lunar Dance'] = pokemon_learnsets_df['Lunar Dance'] + pokemon_learnsets_df['Healing Wish']
pokemon_learnsets_df.drop('Healing Wish', inplace=True, axis=1)
pokemon_learnsets_df['Moonlight'] = pokemon_learnsets_df['Moonlight'] + pokemon_learnsets_df['Morning Sun'] + pokemon_learnsets_df['Synthesis']
pokemon_learnsets_df.drop(['Morning Sun', 'Synthesis'], inplace=True, axis=1)
pokemon_learnsets_df['Recover'] = pokemon_learnsets_df['Recover'] + pokemon_learnsets_df['Soft-Boiled'] + pokemon_learnsets_df['Milk Drink'] + pokemon_learnsets_df['Slack Off']
pokemon_learnsets_df.drop(['Soft-Boiled', 'Milk Drink', 'Slack Off'], inplace=True, axis=1)
pokemon_learnsets_df['Trick'] = pokemon_learnsets_df['Trick'] + pokemon_learnsets_df['Switcheroo']
pokemon_learnsets_df.drop('Switcheroo', inplace=True, axis=1)

pokemon_learnsets_df.shape

We've shrunk it down to 629, but that's not even close to the decrease down to less than 100 columns we are looking for, so we will need a far more clever method.

Also, since we just added together one-hot encoded columns, some of them might have values that aren't 1 or 0 anymore if a pokemon has more than one of the duplicate moves in its moveset. We need to check if that is the case, and then fix it if it is.

In [ ]:
overcounted_moves = pokemon_learnsets_df[pokemon_learnsets_df > 1].sum()
overcounted_moves.value_counts()

In [ ]:
overcounted_moves[overcounted_moves > 1].index

Some of our duplicate moves were indeed double counted. This is easy to fix:

In [ ]:
pokemon_learnsets_df.where(pokemon_learnsets_df <= 1, 1, inplace=True)
overcounted_moves = pokemon_learnsets_df[pokemon_learnsets_df > 1].sum()
overcounted_moves.value_counts()

<a id="explain_cindex"></a>
### Explaining the cindex feature to shrink moves and abilities

There are several requirements that have to be met by any feature we engineer for moves:
- it should be able to be visualized and investigated for purposes of this idea, ideally numerically
- it should be able to be aggregated in such a way that I can dramatically lower the amount of move features

Furthermore, we have two main pieces of data containing move information from which to work from:
- pokemon_learnsets_df, which we just cleaned, which contains one-hot encoded features for all the moves (columns) in the learnset of each pokemon (rows)
- strategies_dict, which is a json that contains all the competitively used strategies for each pokemon (and pokemon many don't have a competitive strategy since they aren't good enough to be used competitively) in addition to which moves and abilities are used in those strategies, since pokemon have many moves in their learnset but can only select 4 moves at a time (and they can select only 1 ability at a time out of the several ability choices they usually have)

This leads to a pretty natural idea for a feature: for each move, we can create a "**competitiveness index**", or "**cindex**", which is the number of pokemon who use each move competitively (since each only gets to choose 4 moves at time, that's a quite selective decision) divided by the number of pokemon who have that move available in their learnset. The more a move is used as a percentage of how much it is available, the more likely that move is to be competitively effective and to have predictive power on the competitive viability of a pokemon. Since the "cindex" is a number, it also meets the two requirements we set out: it can be visualized, numerically investigated and aggregated.

What's even better about this cindex feature is that it works for abilities based on the same principles and because we have analogous data available for abilities (pokemon_abilities_df contains one-hot encoded abilities available for each pokemon and strategies_dict also contains the abilities used for competitive strategies in addition to the moves) and need to dramatically shrink them in a similar way to moves, that makes our task so much easier. We'll start with moves for the sake of continuity.

There is a small, easily fixable issue with combining the information from our pokemon_learnsets_df with strategies_dict to create the new cindex feature: strategies_dict has not had its duplicate moves removed the way that pokemon_learnsets_df has, and since strategies_dict is a complicated json structure, it's probably not that practical to bother fixing in every instance. The solution is just to make a dataframe containing the duplicate moves, and then when one of these moves shows up when looping through strategies_dict, use some code to consider it the same move that the duplicate move was aggregated under (we'll name "collapse" the column that contains the move to aggregate each duplicate move under)

In [ ]:
dup_moves = ['Acid Armor', 'Iron Defense', 'Rock Polish', 'Agility', 'Charm', 'Feather Dance', 'Smokescreen', 'Sand Attack', 'Heal Pulse', 'Floral Healing', 'Leer', 'Tail Whip', 'Lock-On', 'Mind Reader', 'Mean Look', 'Block', 'Healing Wish', 'Lunar Dance', 'Moonlight', 'Morning Sun', 'Synthesis', 'Recover', 'Soft-Boiled', 'Milk Drink', 'Slack Off', 'Trick', 'Switcheroo']

dup_moves_df = pd.DataFrame(index=dup_moves, columns=['collapse'])
dup_moves_df.loc[['Acid Armor', 'Iron Defense'], 'collapse'] = 'Acid Armor'
dup_moves_df.loc[['Rock Polish', 'Agility'], 'collapse'] = 'Agility'
dup_moves_df.loc[['Charm', 'Feather Dance'], 'collapse'] = 'Charm'
dup_moves_df.loc[['Smokescreen', 'Sand Attack'], 'collapse'] = 'Smokescreen'
dup_moves_df.loc[['Heal Pulse', 'Floral Healing'], 'collapse'] = 'Heal Pulse'
dup_moves_df.loc[['Leer', 'Tail Whip'], 'collapse'] = 'Leer'
dup_moves_df.loc[['Lock-On', 'Mind Reader'], 'collapse'] = 'Lock-On'
dup_moves_df.loc[['Mean Look', 'Block'], 'collapse'] = 'Block'
dup_moves_df.loc[['Healing Wish', 'Lunar Dance'], 'collapse'] = 'Lunar Dance'
dup_moves_df.loc[['Moonlight', 'Morning Sun', 'Synthesis'], 'collapse'] = 'Moonlight'
dup_moves_df.loc[['Recover', 'Soft-Boiled', 'Milk Drink', 'Slack Off'], 'collapse'] = 'Recover'
dup_moves_df.loc[['Trick', 'Switcheroo'], 'collapse'] = 'Trick'

dup_moves_df

<a id="dup_abilities"></a>
### Handling duplicate abilities

Now, before we forget, let's find all of the duplicate abilities and give them the same treatment that we did the duplicate moves, using this resource: https://bulbapedia.bulbagarden.net/wiki/Ability_variations

Identical Ability List:
- Air Lock, Cloud Nine
- Battle Armor, Shell Armor
- Clear Body, White Smoke, Full Metal Body
- Filter, Solid Rock, Prism Armor
- Gooey, Tangling Hair
- Insomnia, Vital Spirit
- Mold Breaker, Teravolt, Turboblaze
- Moxie, Chilling Neigh
- Multiscale, Shadow Shield
- Propeller Tail, Stalwart
- Rough Skin, Iron Barbs
- Wimp Out, Emergency Exit
- Arena Trap, Magnet Pull, Shadow Tag (i.e. trapper abilities)
- Natural Cure, Shed Skin (i.e. status absorber abilities)

In [ ]:
pokemon_abilities_df.shape

In [ ]:
pokemon_abilities_df['Air Lock'] = pokemon_abilities_df['Air Lock'] + pokemon_abilities_df['Cloud Nine']
pokemon_abilities_df.drop('Cloud Nine', inplace=True, axis=1)
pokemon_abilities_df['Shell Armor'] = pokemon_abilities_df['Battle Armor'] + pokemon_abilities_df['Shell Armor']
pokemon_abilities_df.drop('Battle Armor', inplace=True, axis=1)
pokemon_abilities_df['Clear Body'] = pokemon_abilities_df['Clear Body'] + pokemon_abilities_df['White Smoke'] + pokemon_abilities_df['Full Metal Body']
pokemon_abilities_df.drop(['White Smoke', 'Full Metal Body'], inplace=True, axis=1)
pokemon_abilities_df['Filter'] = pokemon_abilities_df['Filter'] + pokemon_abilities_df['Solid Rock'] + pokemon_abilities_df['Prism Armor']
pokemon_abilities_df.drop(['Solid Rock', 'Prism Armor'], inplace=True, axis=1)
pokemon_abilities_df['Gooey'] = pokemon_abilities_df['Gooey'] + pokemon_abilities_df['Tangling Hair']
pokemon_abilities_df.drop('Tangling Hair', inplace=True, axis=1)
pokemon_abilities_df['Insomnia'] = pokemon_abilities_df['Insomnia'] + pokemon_abilities_df['Vital Spirit']
pokemon_abilities_df.drop('Vital Spirit', inplace=True, axis=1)
pokemon_abilities_df['Teravolt'] = pokemon_abilities_df['Mold Breaker'] + pokemon_abilities_df['Teravolt'] + pokemon_abilities_df['Turboblaze']
pokemon_abilities_df.drop(['Mold Breaker', 'Turboblaze'], inplace=True, axis=1)
pokemon_abilities_df['Moxie'] = pokemon_abilities_df['Moxie'] + pokemon_abilities_df['Chilling Neigh']
pokemon_abilities_df.drop('Chilling Neigh', inplace=True, axis=1)
pokemon_abilities_df['Multiscale'] = pokemon_abilities_df['Multiscale'] + pokemon_abilities_df['Shadow Shield']
pokemon_abilities_df.drop('Shadow Shield', inplace=True, axis=1)
pokemon_abilities_df['Stalwart'] = pokemon_abilities_df['Propeller Tail'] + pokemon_abilities_df['Stalwart']
pokemon_abilities_df.drop('Propeller Tail', inplace=True, axis=1)
pokemon_abilities_df['Rough Skin'] = pokemon_abilities_df['Rough Skin'] + pokemon_abilities_df['Iron Barbs']
pokemon_abilities_df.drop('Iron Barbs', inplace=True, axis=1)
pokemon_abilities_df['Wimp Out'] = pokemon_abilities_df['Wimp Out'] + pokemon_abilities_df['Emergency Exit']
pokemon_abilities_df.drop('Emergency Exit', inplace=True, axis=1)
#Trapper abilities: Arena Trap, Magnet Pull, Shadow Tag
pokemon_abilities_df['Trapper'] = pokemon_abilities_df['Arena Trap'] + pokemon_abilities_df['Magnet Pull'] + pokemon_abilities_df['Shadow Tag']
pokemon_abilities_df.drop(['Arena Trap', 'Magnet Pull', 'Shadow Tag'], inplace=True, axis=1)
#Status Absorber abilities: Natural Cure, Shed Skin
pokemon_abilities_df['Status Absorber'] = pokemon_abilities_df['Natural Cure'] + pokemon_abilities_df['Shed Skin']
pokemon_abilities_df.drop(['Natural Cure', 'Shed Skin'], inplace=True, axis=1)

pokemon_abilities_df.shape

As with moves, it gives a minor shrinkage in the columns, which is a sign that it worked properly.

Just like with moves, to take care of any pokemon who may be double counting these abilities due to having more than one of the same duplicate abilities, we can just do this:

In [ ]:
pokemon_abilities_df.where(pokemon_abilities_df <= 1, 1, inplace=True)
overcounted_abilities = pokemon_abilities_df[pokemon_abilities_df > 1].sum()
overcounted_abilities.value_counts()

And as with moves, we need to make a dataframe of our duplicate abilities with a column containing the heading each was collapsed under, so that we can recognize duplicate abilities in strategies_dict when using it to make the cindex feature for abilities:

In [ ]:
dup_abilities= ['Air Lock', 'Cloud Nine', 'Battle Armor', 'Shell Armor', 'Clear Body', 'White Smoke', 'Full Metal Body', 'Filter', 'Solid Rock', 'Prism Armor', 'Gooey', 'Tangling Hair', 'Insomnia', 'Vital Spirit', 'Mold Breaker', 'Teravolt', 'Turboblaze', 'Moxie', 'Chilling Neigh', 'Multiscale', 'Shadow Shield', 'Propeller Tail', 'Stalwart', 'Rough Skin', 'Iron Barbs', 'Wimp Out', 'Emergency Exit', 'Arena Trap', 'Magnet Pull', 'Shadow Tag', 'Natural Cure', 'Shed Skin']

dup_abilities_df = pd.DataFrame(index=dup_abilities, columns=['collapse'])
dup_abilities_df.loc[['Air Lock', 'Cloud Nine'], 'collapse'] = 'Air Lock'
dup_abilities_df.loc[['Battle Armor', 'Shell Armor'], 'collapse'] = 'Shell Armor'
dup_abilities_df.loc[['Clear Body', 'White Smoke', 'Full Metal Body'], 'collapse'] = 'Clear Body'
dup_abilities_df.loc[['Filter', 'Solid Rock', 'Prism Armor'], 'collapse'] = 'Filter'
dup_abilities_df.loc[['Gooey', 'Tangling Hair'], 'collapse'] = 'Gooey'
dup_abilities_df.loc[['Insomnia', 'Vital Spirit'], 'collapse'] = 'Insomnia'
dup_abilities_df.loc[['Mold Breaker', 'Teravolt', 'Turboblaze'], 'collapse'] = 'Teravolt'
dup_abilities_df.loc[['Moxie', 'Chilling Neigh'], 'collapse'] = 'Moxie'
dup_abilities_df.loc[['Multiscale', 'Shadow Shield'], 'collapse'] = 'Multiscale'
dup_abilities_df.loc[['Propeller Tail', 'Stalwart'], 'collapse'] = 'Stalwart'
dup_abilities_df.loc[['Rough Skin', 'Iron Barbs'], 'collapse'] = 'Rough Skin'
dup_abilities_df.loc[['Wimp Out', 'Emergency Exit'], 'collapse'] = 'Wimp Out'
#Trapper abilities: Arena Trap, Magnet Pull, Shadow Tag
dup_abilities_df.loc[['Arena Trap', 'Magnet Pull', 'Shadow Tag'], 'collapse'] = 'Trapper'
#Status Absorber abilities: Natural Cure, Shed Skin
dup_abilities_df.loc[['Natural Cure', 'Shed Skin'], 'collapse'] = 'Status Absorber'

dup_abilities_df

<a id="pca_throw"></a>
### Throwing the data into PCA

Before we get started creating this new complicated cindex feature, it's probably a good idea to throw all of our data into PCA just once, in order to see if any major patterns are noticed. I have an intuition from this article that using PCA on what is primarily hundreds of one-hot encoded features isn't going to be very useful: https://towardsdatascience.com/pca-is-not-feature-selection-3344fb764ae6

But since it's easy and quick, there's little reason not to give it a try.  First we need to merge together all of our data so far:

In [ ]:
PCAattempt1_df = pokemon_df.merge(pokemon_types_summary, on='name')
PCAattempt1_df = PCAattempt1_df.merge(pokemon_abilities_df, on='name')
PCAattempt1_df = PCAattempt1_df.merge(pokemon_learnsets_df, on='name')
PCAattempt1_df.shape

That's a total of 870 columns, which is still more than the number of pokemon we have so our model has literally 0 degrees of freedom.

In [ ]:
PCAattempt1_nonnumeric = PCAattempt1_df.select_dtypes(include='object', exclude='number')
PCAattempt1_numeric = PCAattempt1_df.select_dtypes(include='number')
PCAattempt1_columns = PCAattempt1_numeric.columns
PCAattempt1_scaled = scale(PCAattempt1_numeric)
PCAattempt1_df_scaled = pd.DataFrame(PCAattempt1_scaled, index=pokemon_df.index, columns=PCAattempt1_columns)
PCAattempt1_fit = PCA().fit(PCAattempt1_df_scaled)

plt.subplots(figsize=(10, 6))
plt.plot(PCAattempt1_fit.explained_variance_ratio_.cumsum())
plt.xlabel('Component #')
plt.ylabel('Cumulative ratio variance')
plt.title('Cumulative variance ratio explained by PCA components for 868 pokemon stats');

Considering that about 100 components are necessary to explain 60% of the variance, PCA will certainly not be useful given the current state of the data, confirming my intuition and setting us on our way to creating the more clever cindex feature:

<a id="moves_cindex_df"></a>
### Creating the moves_cindex dataframe

To start, we need to make a new json/dictionary to look through, derived from strategies_dict but which has pokemon that don't have any competitive strategies removed.

The "cindex" value for the moves will be calculated as the number of pokemon who use the move competitively (out of the only 4 moves they can use at a time), divided by the number of pokemon who have the move in their learnset. The only way this will work though is if we only consider pokemon who have at least one competitive learnset, because if we divide by the total number of pokemon who have the move in their learnset (including one's who don't have competitive strategies at all, which may be for other reasons than the moves available to them, such as their low stats), then we will be dividing by a larger number than the pokemon we really considered competitive movesets from, which will lead to artificially low numbers for the cindex. Thus, we also need to aggregate the pokemon without competitive strategies and remove them from the calculation (though they can still be considered for cindex features later since they have learnsets of moves, thus we can still evaluate the quality of their moves based on what we find).

In [ ]:
cindex_dict = strategies_dict.copy()
pokemon_noncompetitive = []

for pokemon, formatlist in strategies_dict.items():
    if formatlist == []:
        pokemon_noncompetitive.append(pokemon)
        cindex_dict.pop(pokemon)
    else:
        current_pokemon_formats = []
        for formatdict in formatlist:
            formatkey = next(iter(formatdict))
            if formatkey in ranked_formats:
                current_pokemon_formats.append(formatkey)
        if current_pokemon_formats == []:
            pokemon_noncompetitive.append(pokemon)
            cindex_dict.pop(pokemon)

Also, a quick reminder: if you want to better understand the structure of json and thus why I made the code decision I did above, then I recommend putting the contents of the strategies_dict json file into http://jsonviewer.stack.hu/.  Then it will be more obvious why I executed loops, lists, etc. in the way that I chose to, and the code will be less esoteric.

Now we will create the moves_cindex dataframe! The index will be the names of all the moves (contained in the columns of pokemon_learnsets_df), and three columns we want for certain are:
- n_learnset (the number of times each move is in the learnset of pokemon with competitive strategies)
- n_competitive (the number of times each move is actually used competitively by pokemon with competitive strategies)
- cindex (the main purpose of this dataframe, (n_competitive/n_learnset) x 100 to make it a percent value

Let's look at moves_df to see if there any other features that might be useful to include in this dataframe:

In [ ]:
moves_df.head()

- Power is very important to how strong a move is, so will also be a useful numerical feature to work with when understanding how strong the learnset of a pokemon is
- Type is also important to understand the range of "type coverage" each pokemon (the number of types they have available good moves for)
- Category is important (either physical, special, or non-damaging), since we may want to investigate moves separately based on those categories

I can't imagine using the other categories very much, because even though things like accuracy and pp are important in pokemon, they hardly make a move good or bad by themselves (maybe more bad than good)

In [ ]:
moves_cindex = pd.DataFrame(0, index=pokemon_learnsets_df.columns, columns=['n_learnset', 'n_competitive', 'cindex'])

for pokemon, formatlist in cindex_dict.items():
    already_checked = []
    for formatdict in formatlist:
        formatkey = next(iter(formatdict))
        if formatkey not in ranked_formats:
            continue
        for movesetlist in formatdict.values():
            for moveset in movesetlist:
                moveset_name = [name for name in moveset.keys() if name != 'abilities'][0]
                for move in moveset[moveset_name]:
                    if move in dup_moves:
                        move = dup_moves_df.loc[move, 'collapse']
                    if move not in already_checked:
                        moves_cindex.loc[move, 'n_competitive'] += 1
                        already_checked.append(move)
                        
for move in pokemon_learnsets_df.columns:
    moves_cindex.loc[move, 'n_learnset'] = pokemon_learnsets_df.drop(index=pokemon_noncompetitive)[move].sum()
    
for move in moves_cindex.index:
    moves_cindex.loc[move, 'power'] = moves_df.loc[move, 'power']
    moves_cindex.loc[move, 'type'] = moves_df.loc[move, 'type']
    moves_cindex.loc[move, 'category'] = moves_df.loc[move, 'category']
    
moves_cindex.head()

Upon futher investigation in [EDA_experiments](./EDA_experiments.ipynb#Unique_Shield) (the Unique Shield section in the table of contents if the link doesn't take you there directly), I found 4 moves (King's Shield, Spiky Shield, Obstruct, Baneful Bunker), all uniquely more powerful versions of the move "Protect" due to their additional effects, to have a more understandable cindex as a single move, since these moves are all only contained by one pokemon, so their cindex alone is unrealistically dramatic (either 100 or 0), but together it is 75 which is more realistic and interpretable due to the increased n_learnset size of 4 (essentially increasing the sample size).

In [ ]:
#Unique Shield moves: King's Shield, Spiky Shield, Obstruct, Baneful Bunker
moves_cindex.loc['Unique Shield', ['n_learnset', 'n_competitive']] = moves_cindex.loc[['King\'s Shield', 'Spiky Shield', 'Obstruct', 'Baneful Bunker'], ['n_learnset', 'n_competitive']].sum()
moves_cindex.loc['Unique Shield', 'power'] = 0.0
moves_cindex.loc['Unique Shield', 'type'] = 'Normal' #similar to protect
moves_cindex.loc['Unique Shield', 'category'] = 'Non-Damaging'
moves_cindex.drop(index=['King\'s Shield', 'Spiky Shield', 'Obstruct', 'Baneful Bunker'], inplace=True)
    
moves_cindex.loc['Unique Shield', :]

Now we can populate the dataframe with calculations of the cindex using the n_learnset and n_competitive features:

In [ ]:
moves_cindex['cindex'] = 100*(moves_cindex['n_competitive'] / moves_cindex['n_learnset'])

moves_cindex.head(10)

<a id="abilities_cindex_df"></a>
### Creating the abilities_cindex dataframe

Fortunately this will be even easier than creating moves_cindex, because cindex_dict and pokemon_noncompetitive are already created for us.  Furthermore:

In [ ]:
abilities_df.head()

abilities_df doesn't have any other interesting features about abilities that we need to save, like power, type or category, so we really only have the 3 core columns (n_abilityset, the equivalent of n_learnset last time, and n_competitive and cindex, which serve exactly the same purposes as last time)

In [ ]:
abilities_cindex = pd.DataFrame(0, index=pokemon_abilities_df.columns, columns=['n_abilityset', 'n_competitive', 'cindex'])

for pokemon, formatlist in cindex_dict.items():
    already_checked = []
    for formatdict in formatlist:
        formatkey = next(iter(formatdict))
        if formatkey not in ranked_formats:
            continue
        for movesetlist in formatdict.values():
            for moveset in movesetlist:
                for ability in moveset['abilities']:
                    if ability in dup_abilities:
                        ability = dup_abilities_df.loc[ability, 'collapse']
                    if ability not in already_checked:
                        abilities_cindex.loc[ability, 'n_competitive'] += 1
                        already_checked.append(ability)
                        
for ability in pokemon_abilities_df.columns:
    abilities_cindex.loc[ability, 'n_abilityset'] = pokemon_abilities_df.drop(index=pokemon_noncompetitive)[ability].sum()

abilities_cindex.head()

And we need to add the cindex feature to replace all the 0's:

In [ ]:
abilities_cindex['cindex'] = 100*(abilities_cindex['n_competitive'] / abilities_cindex['n_abilityset'])
abilities_cindex.head()

As with moves_cindex, there was at least one slight bug I encountered:

In [ ]:
abilities_cindex[abilities_cindex['cindex'] > 100]

We have an ability with above 100 cindex! That's technically impossible, so it must have just proceeded from a mistake or something we overlooked, and it did:

Zygarde-10% (a UU tiered pokemon) technically has the Power Construct ability along with Zygarde (an Uber pokemon which is closely related). The reason I removed it from Zygarde-10% is because the ability is only allowed to be used in Ubers, where Zygarde is used instead of Zygarde-10%. But in strategies_dict, I didn't go through and change that minor detail, and Zygarde-10% does have an Uber moveset even though it isn't really viable there so it's really a UU pokemon which was manually kept at that level when I created tierchange_df. So, I just need to manually change this ability to its appropriate n_competitive (which comes from strategies_dict), namely changing it back to 1 instead of 2, then re-do the calculations and everything should be fixed.

In [ ]:
abilities_cindex.loc['Power Construct', 'n_competitive'] = 1
abilities_cindex['cindex'] = 100*(abilities_cindex['n_competitive'] / abilities_cindex['n_abilityset'])
abilities_cindex.head()

<a id="role_features"></a>
### Investigating Roles for Features

Before, we start investigating moves_cindex and using it for aggregate features, it's best to see if there are any more distinctive features I want to create first, and there are a lot I want to try. Since many of these moves will be distinct features, they don't need to be treated as moves in moves_cindex which have to be aggregated, so that way we can take out very important things to be sure they get noticed and not lost in the aggregation, while also making our aggregation more focused.

To see which roles are typically used competitively in pokemon, I looked at these resources (the role compendiums for each tier in competitive pokemon and which pokemon typically play those roles) which gave great information:

- https://www.smogon.com/forums/threads/ss-ou-role-compendium.3657644/
- https://www.smogon.com/forums/threads/ss-uu-role-compendium-20-07-2021-update-see-post-50.3659197/
- https://www.smogon.com/forums/threads/ss-ru-role-compendium.3660895/
- https://www.smogon.com/forums/threads/ss-nu-role-compendium-updated-09-2021.3664555/
- https://www.smogon.com/forums/threads/ss-pu-resources-thread-sample-submissions-open.3662279/#post-8421304
- https://www.smogon.com/forums/threads/ss-zu-resources-hub.3665776/#post-8502699

The 4 broader roles are Utility, Offensive, Defensive and Weather.

Within each of those there are a lot of specific roles that we can learn about in such a way that we could turn them into features!

**Utility breaks down into:**
- Entry Hazards
- Hazard Control
- Clerics and Wish
- Other

Utility: Entry Hazards
- Spikes
- Stealth Rock
- Sticky Web
- Toxic Spikes
- Suicide Lead

Utility: Hazard Control
- Defog
- Magic Bounce
- Rapid Spin
- Spinblocker
- Defog Deterrant

Utility: Clerics and Wish
- Heal Bell/Aromatherapy/Jungle Healing
- Healing Wish/Lunar Dance
- Wish

Utility: Other
- Knock Off/Corrosive Gas
- Switcheroo/Trick
- Screen Setter
- Aurora Veil (may be something like screen setter, or weather abuse)
- Status Absorber (Natural Cure, Shed Skin, etc.)
- Unaware
- VoltTurn (Flip Turn, U-turn, or Volt Switch)
- Teleport
- Parting Shot (may fit "defensive pivot" with teleport, but VoltTurn also can be, I might even put all 5 of those together though some are obviously better so not sure)
- Trapper (Anchor Shot, Spirit Shackle, Magnet Pull)
- Taunt (may relate to things like Suicide Lead and Stallbreaker)
- Substitute
- Phazer (Roar, Dragon Tail, Whirlwind, Circle Throw)
- Memento (could arguably go with Healing Wish, as a sacrifice move)
- Toxic
- Will-o-wisp
- Paralysis/Thunder Wave
- Sleep
- Encore
- Tailwind
- Haze/Clear Smog

Offensive:
- Physical Wallbreaker
- Special Wallbreaker (mixed is not necessary since it's represented by any pokemon high on both)
- Choice items (these may not be necessary as roles, because items are to be inferred rather than directly used, and otherwise these pokemon are usually just sweepers, except maybe the scarf, maybe split into choice attacker vs doing something else with the choice [usually scarf])
- Setup Sweepers (need to space out these moves, some may be defensive)
- Priority users (may need to split this up, including some maybe in weather, and considering abilities like Technician)
- Stallbreaker (not sure what this will have: usually taunt and setup it seems, and setup may be a separate thing so Taunt may already capture this, maybe perish song as well and other phazing moves, though those are also stalling tactics so not sure, definitely taunt though)

Defensive:
- Physical Wall/Tank
- Special Wall/Tank

Weather:
- Weather/Terrain Setters
- Weather/Terrain Speedsters
- Weather Advantage? (Sand Veil, other abusers?, maybe make this into a stat that makes speeders a 2 and pokemon with weather advantage moves/type as a 1, or maybe even up to 3 taking into account type, moveset and speed gain and maybe other advantages as well)
- Trick Room

One other thing that would be useful is a dictionary of the competitive roles and a list of their names from strategies_dict. Although it is not as well organized and split neatly into the roles we're actually going to use above, it still may offer some insight.

In [ ]:
roles_dict = {}
for pokemon, formatlist in strategies_dict.items():
    for formatdict in formatlist:
        formatkey = next(iter(formatdict))
        if formatkey not in ranked_formats:
            continue
        for movesetlist in formatdict.values():
            for moveset in movesetlist:
                role_dict = {}
                role_dict['name'] = pokemon
                role_dict['format'] = formatkey
                role_dict['abilities'] = moveset['abilities']
                role = [name for name in moveset.keys() if name != 'abilities'][0]
                role_dict['moveset'] = moveset[role]
                if role not in roles_dict.keys():
                    roles_dict[role] = [role_dict]
                else:
                    roles_dict[role].append(role_dict)
                    
roles_list = sorted(list(roles_dict.keys()))
roles_list

And finally, we need a dataframe to hold the new features we're going to make out of these roles. Fittingly, we'll call it shrinkfeatures_df. We'll also check out the percentiles of the "cindex" column of moves_cindex so that we can recognize a good (and not-so-good) move when we see one (and the same for abilities_cindex).

In [ ]:
shrinkfeatures_df = pd.DataFrame(index=pokemon_df.index)

moves_cindex['cindex'].describe(percentiles = [n/20 for n in range(1, 20)])

In [ ]:
abilities_cindex['cindex'].describe(percentiles = [n/20 for n in range(1, 20)])

<a id="utility_entry"></a>
### Utility: Entry Hazards

- Spikes
- Stealth Rock
- Sticky Web
- Toxic Spikes
- Suicide Lead

Entry Hazards are mostly moves, so I have the choice to represent each move as a separate feature, to combine them into a single one-hot encoded feature, or to combine them into a single feature with values 0-4 (based on how many entry hazards a pokemon has or ranked based on how good their best hazard is).

Suicide Lead is trickier since it's not a move, and I should probably investigate it to see what it implies.

So first I'll look at what the role list has to say about these moves, then examine them in moves_cindex and the role_dict. The only Entry Hazard without a uniquely listed role in roles_list is Toxic Spikes.  But first let's look at them in moves_cindex:

In [ ]:
moves_cindex.loc[['Spikes', 'Toxic Spikes', 'Stealth Rock', 'Sticky Web'], :]

These are all pretty great moves, above 85 percentile for all and some far above that. I originally planned on making the entry hazard hazard based on how many of those 4 moves the pokemon has, but the slight problem is that Stealth Rock has a strong correlation with the rock type, so the feature ended up being largely dominated by that since Stealth Rock is in so many more learnsets than the other entry hazards (and while Stealth Rock is very good, it isn't the best on there by far). To restrict Stealth Rock's rock correlation from dominating the feature, I found it better to make the feature into a hierarchy where 0 means having none of the moves, 1 means having toxic spikes, 2 means having Stealth Rock, 3 means having Spikes and 4 means having Sticky Web (and having played against Sticky Web in Pokemon Showdown personally, I can see why it's the highest one on here, it can easily win games if not checked).

The score for each pokemon on this feature will just be the highest one they have, so we'll input the lower values first so that the higher values replace the lower one's if the pokemon has both.

In [ ]:
#Utility: Entry Hazards
shrinkfeatures_df['Entry Hazards'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Toxic Spikes'] == 1, 'Entry Hazards'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Stealth Rock'] == 1, 'Entry Hazards'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Spikes'] == 1, 'Entry Hazards'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Sticky Web'] == 1, 'Entry Hazards'] = 4
moves_cindex.drop(index=['Spikes', 'Toxic Spikes', 'Stealth Rock', 'Sticky Web'], inplace=True)

shrinkfeatures_df['Entry Hazards'].value_counts()

Now let's use roles_dict to investigate Suicide Lead:

In [ ]:
roles_dict['Suicide Lead']

What's notable here is that all of these pokemon have other entry hazard moves, so "Suicide Lead" probably doesn't need to have its own separate feature.  

There are some moves like Explosion, high recoil moves (e.g. Flare Blitz, Head Smash), Final Gambit and Destiny Bond which are used with high frequency, but they don't seem very common so it's hard to consider them as a feature for entry hazard even though they might be individually good in these cases.  Encore, Yawn and Infestation are something different.

There is one more notable fact though: almost all of these pokemon have Taunt.  So we have to consider whether including that as a feature or as part of  the Entry Hazard kit.

Let's investigate these moves in the cindex:

In [ ]:
moves_cindex.loc[['Explosion', 'Flare Blitz', 'Destiny Bond', 'Head Smash', 'Final Gambit', 'Taunt'], :]

None of these moves seem important to make special features. Flare Blitz is a generally quite strong attack so no wonder Mew uses it. Explosion, Head Smash, Final Gambit and Destiny Bond are competitive but only decent moves, certainly nothing that could justify their own feature. Overall those moves can all be ignored as separate roles for now.

Taunt is used with particularly high frequency though, which is the notable thing about it, in spite of only being only in the 25% of moves (keep in mind that about half of moves are complete garbage competitively), which makes it a middling competitive move.  Since we want to investigate Hazard Lead anyways, it would be interesting to see if Taunt shows up there to so we can see how unique it is to Suicide Lead:

In [ ]:
roles_dict['Hazard Lead']

We again see Froslass with Taunt, using Destiny Bond to boot, but Destiny Bond seems uniquely useful to that pokemon so doesn't seem justifiable to do anything special with. Excadrill has no Taunt though. Maybe we should look at some other entry hazard roles and see how much Taunt they have:

In [ ]:
roles_dict['Spikes']

In [ ]:
roles_dict['Stallbreaker']

I also looked at Utility and Stealth Rocks, which you can view in [EDA_experiments](./EDA_experiments.ipynb#utility_stealth_rocks) in the "utility and stealth rocks in roles_dict" section if you want since they were unnecessarily lengthy for this more polished notebook. Overall, it's used a lot in these entry hazard roles, but about half the time or even less.

There are some problems with Taunt as a separate role though, namely a not super high cindex (though not low of course), high correlation with the dark type, plus only sometimes being used in these utility roles. It wasn't significant enough as a feature, so I later decided to include in the aggregation of miscellaneous status moves.

<a id="hazard_control"></a>
### Utility: Hazard Control

- Defog
- Magic Bounce
- Rapid Spin
- Spinblocker
- Defog Deterrant

Defog and Rapid Spin are moves, Magic Bounce is an ability and Defog Deterrant generally comes down to specific abilities (Defiant/Contrary/Competitive), and Spinblocker is a side effect of being Ghost type.

In [ ]:
moves_cindex.loc[['Defog', 'Rapid Spin'], :]

In [ ]:
abilities_cindex.loc[['Magic Bounce', 'Defiant', 'Contrary', 'Competitive'], :]

I think I'm going to make two features here.

- The 1st is "Hazard Removal", which will include Rapid Spin, Defog and Magic Bounce. Although Magic Bounce doesn't technically remove hazards, it makes setting them impossible and even throws them back at the opponent on a switch in, and it's a passive ability which is always active as opposed to Defog and Rapid Spin which you have to waste a turn using.  Magic Bounce is a very rare and powerful ability so it deserves to be a "2" on this feature, whereas having Defog or Rapid Spin will be a "1"

- The 2nd is "Removal Deterrent", which are the 3 other abilities which deter people from removing hazards that have ALREADY been set, rather than removing or deterring the new setting of hazards. Any pokemon with those abilities will have a 2 in the feature, whereas any pokemon with a ghost type will have a 1 since they can block Rapid Spin (since Rapid Spin is rarer than Defog, being ghost type isn't as good as those abilities, plus if I set ghost on an equal level to the abilities, then this feature just ends up correlating with ghost type which makes it less useful)

In [ ]:
#Utility: Hazard Control (pd.where is being used to one-hot encode here)
shrinkfeatures_df['Hazard Removal'] = pokemon_learnsets_df.loc[:, ['Rapid Spin', 'Defog']].sum(axis=1)
shrinkfeatures_df.loc[:, 'Hazard Removal'].where(shrinkfeatures_df['Hazard Removal'] <= 1, 1, inplace=True)
moves_cindex.drop(index=['Rapid Spin', 'Defog'], inplace=True)
shrinkfeatures_df.loc[pokemon_abilities_df['Magic Bounce'] == 1, 'Hazard Removal'] = 2

shrinkfeatures_df['Hazard Removal'].value_counts()

In [ ]:
shrinkfeatures_df['Removal Deterrent'] = 0
shrinkfeatures_df.loc[pokemon_types_df['Ghost'] == 1, 'Removal Deterrent'] = 1
shrinkfeatures_df.loc[pokemon_abilities_df['Defiant'] == 1, 'Removal Deterrent'] = 2
shrinkfeatures_df.loc[pokemon_abilities_df['Contrary'] == 1, 'Removal Deterrent'] = 2
shrinkfeatures_df.loc[pokemon_abilities_df['Competitive'] == 1, 'Removal Deterrent'] = 2

shrinkfeatures_df['Removal Deterrent'].value_counts()

<a id="utility_clerics"></a>
### Utility: Clerics and Wish

- Heal Bell/Aromatherapy/Jungle Healing
- Healing Wish/Lunar Dance/Memento
- Wish

I realize now that I may have made a slight mistake in collapsing Lunar Dance and Healing Wish together as they are VERY similar, but Lunar Dance also heals PP.  While they may relate together in a feature, since I respected the subtle difference in Heal Bell/Aromatherapy, it only seems natural to do the same for Healing Wish and Lunar Dance.  The only problem is that I may have to redo some things.  Not sure though, since changing a single move is unlikely to affect my choices in the analysis above, even though it affects them slightly.  I was going to examine the distros again briefly anyways to find the exact final cutoffs), so it's probably worth fixing this before I do that (for now I'll just leave it)

In [ ]:
moves_cindex.loc[['Heal Bell', 'Aromatherapy', 'Jungle Healing', 'Lunar Dance', 'Memento', 'Wish'], :]

In [ ]:
roles_dict['Cleric']

This is a trickier decision because these moves aren't generally as good as what we've encountered previously.

Lunar Dance/Healing Wish seems like a powerful tactic, and Wish also seems like a common and useful kit.  Heal Bell/Aromatherapy are borderline and often seem to be used alongside Wish (and thus in many cases don't contain much information), either that or it's only an option alongside other moves. Jungle Healing is another rather unique move in that it doesn't heal as much as the normal recovery options like Recover or Roost, but acts as Heal Bell or Aromatherapy (so I may include it in that feature, but as a "2" instead of a "1"), since it acts in a similar way to wish and is just superior to heal bell/aromatherapy

There is another option for Heal Bell/Aromatherapy, which is having a one one-hot encoded feature for having at least one of those, another for having wish, and another for having healing wish/lunar dance, and then combine the one-hot encoded features into a single cleric feature from 0 to 3 (and if I aggregate in this way, there's no need to go back and fix Lunar Dance and Healing Wish since they'll be aggregated anyways). These are different moves so having more than one of these cleric moves can make a significant difference in one's ability to play a cleric. Plus, the individual moves probably don't mean a lot alone since they are overall focused on something narrow and not very informative, so it's better to aggregate to get something more meaningful at scale.

Memento is a lot more situational and not as related to the main cleric moves, so it won't be included here, but I will aggregate it into miscellaneous status moves later.

In [ ]:
shrinkfeatures_df['Party Heal'] = pokemon_learnsets_df.loc[:, ['Heal Bell', 'Aromatherapy']].sum(axis=1)
shrinkfeatures_df.loc[:, 'Party Heal'].where(shrinkfeatures_df['Party Heal'] <= 1, 1, inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Jungle Healing'] == 1, 'Party Heal'] = 2
moves_cindex.drop(index=['Heal Bell', 'Aromatherapy', 'Jungle Healing'], inplace=True)

shrinkfeatures_df['Party Heal'].value_counts()

In [ ]:
shrinkfeatures_df['Setup Support'] = pokemon_learnsets_df['Lunar Dance']
moves_cindex.drop(index='Lunar Dance', inplace=True)

shrinkfeatures_df['Setup Support'].value_counts()

In [ ]:
shrinkfeatures_df['Wish'] = pokemon_learnsets_df['Wish']
moves_cindex.drop(index='Wish', inplace=True)

shrinkfeatures_df['Wish'].value_counts()

In [ ]:
#Utility: Clerics and Wish
shrinkfeatures_df['Cleric'] = shrinkfeatures_df.loc[:, ['Party Heal', 'Setup Support', 'Wish']].sum(axis=1)
shrinkfeatures_df.drop(columns=['Party Heal', 'Setup Support', 'Wish'], inplace=True)

shrinkfeatures_df['Cleric'].value_counts()

<a id="utility_other"></a>
### Utility: Other

- Knock Off/Corrosive Gas
- Switcheroo/Trick
- Screen Setter
- Aurora Veil (may be something like screen setter, or weather abuse)
- Status Absorber (Natural Cure, Shed Skin, etc.)
- Unaware
- VoltTurn (Flip Turn, U-turn, or Volt Switch)
- Teleport
- Parting Shot (may fit "defensive pivot" with teleport, but VoltTurn also can be, I might even put all 5 of those together though some are obviously better so not sure)
- Trapper (Anchor Shot, Spirit Shackle, Magnet Pull)
- Substitute
- Phazer (Roar, Dragon Tail, Whirlwind, Circle Throw)
- Toxic
- Will-o-wisp
- Paralysis/Thunder Wave
- Sleep
- Encore
- Tailwind
- Haze/Clear Smog

In [ ]:
moves_cindex.loc[['Knock Off', 'Corrosive Gas', 'Trick', 'Reflect', 'Light Screen', 'Aurora Veil', 'Flip Turn', 'U-turn', 'Volt Switch', 'Teleport', 'Parting Shot', 'Anchor Shot', 'Spirit Shackle', 'Substitute', 'Roar', 'Dragon Tail', 'Whirlwind', 'Circle Throw', 'Toxic', 'Will-O-Wisp', 'Thunder Wave', 'Stun Spore', 'Glare', 'Spore', 'Sing', 'Lovely Kiss', 'Hypnosis', 'Sleep Powder', 'Encore', 'Tailwind', 'Haze', 'Clear Smog', 'Yawn', 'Perish Song'], :].sort_values(by='cindex')

There's a lot going on here so let's analyze it one at a time. We can start with a pivot feature, which I saw a lot when I actually tried playing Pokemon Showdown.

U-Turn, Volt Switch and Flip Turn are all very powerful pivot moves (and all on a pretty similar playing field). Parting Shot is a less powerful pivot move which lowers stats instead of doing damage, but it's still a quality move so deserves to be included in a lower tier of this feature. And Teleport is another pivot move which has no effect on the opponent and purely switches, but it also isn't terrible perhaps since it can offer safe switch-in's when desired. So, as with before we'll have a hierarchy where Teleport is 1, Parting Shot is 2, and the others are 3.

In [ ]:
#Utility: Pivot
shrinkfeatures_df['Pivot'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Teleport'] == 1, 'Pivot'] = 1
moves_cindex.drop(index='Teleport', inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Parting Shot'] == 1, 'Pivot'] = 2
moves_cindex.drop(index='Parting Shot', inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['U-turn'] == 1, 'Pivot'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Volt Switch'] == 1, 'Pivot'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Flip Turn'] == 1, 'Pivot'] = 3
moves_cindex.drop(index=['U-turn', 'Volt Switch', 'Flip Turn'], inplace=True)

shrinkfeatures_df['Pivot'].value_counts()

Next, we have an idea for an item removal feature, which contains Knock Off and Corrosive Gas.  Knock off is a very strong, while Corrosive Gas is only barely competitive (which makes sense since it doesn't do damage, while Knock Off is also a damaging attack with basically the same effect). So Knock Off will be 2 and Corrosive Gas will be 1.

In [ ]:
#Utility: Item Removal
shrinkfeatures_df['Item Removal'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Corrosive Gas'] == 1, 'Item Removal'] = 1
moves_cindex.drop(index='Corrosive Gas', inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Knock Off'] == 1, 'Item Removal'] = 2
moves_cindex.drop(index='Knock Off', inplace=True)

shrinkfeatures_df['Item Removal'].value_counts()

Besides those pretty straightforward features, others are far more questionable.

You could make features around Sleep moves, Burn moves, Poison moves and Paralysis moves, but I tried in the experiments to create a feature for each status condition move a pokemon could use and it didn't go well; the features were very unimportant.

Likewise, other moves like Trick, Light Screen, Reflect, Substitute, Roar, Dragon Tail, Whirlwind, Circle Throw, Encore, Tailwind, Haze, Smog, etc. may occasionally be nice, but they just aren't good enough to be their own feature and have any feature importance in the models (I tested this), but that doesn't mean they should be thrown away entirely.

Some of these moves are problematic for another reason: they are so common (ergo, their n_learnset is so high) that most pokemon have access to the move. And no matter how good or bad such moves are, they give the model nearly no information if most pokemon have access to them, since the model can only work with their static stats like their learnset (it doesn't know anything about which pokemon actually use the move since we can't use this model predictively if we require more than the static stats of the pokemon, since new pokemon will have had no competitive use no matter how good they are). We should get rid of these moves from moves_cindex, and some of those moves are listed above (such as Toxic and Substitute), but we will deal with that later.

Aurora Veil is useful in niche situations, but it's a weather related feature so we will deal with it in that category.

Unaware is a rare and quite middling ability in terms of cindex, so making it a separate feature would be absurd

In [ ]:
#binding moves
moves_cindex.loc[['Bind', 'Sand Tomb', 'Snap Trap', 'Wrap', 'Fire Spin', 'Infestation', 'Magma Storm', 'Thunder Cage', 'Whirlpool'], :].sort_values(by='cindex')

In [ ]:
#trapping moves
moves_cindex.loc[['Anchor Shot', 'Jaw Lock', 'Spirit Shackle', 'Thousand Waves', 'Block', 'Fairy Lock', 'No Retreat', 'Octolock'], :].sort_values(by='cindex')

The only binding and trapping moves that are good are uniquely powerful moves (moves with perfect cindex but very low n_learnset), so they will get their due in the feature for unique powerful moves. The others are garbage and not even close to worth being made into a feature.

Status Absorber and Trapper abilities were already collapsed together in dup_abilities, since they do give some good information together, but they are too niche and rare to make for individual features which have any consistent feature importance in models.

<a id="offense_setup"></a>
### Offensive: Setup

- Physical Wallbreaker
- Special Wallbreaker (mixed is not necessary since it's represented by any pokemon high on both)
- Choice items (these may not be necessary as roles, because items are to be inferred rather than directly used, and otherwise these pokemon are usually just sweepers, except maybe the scarf, maybe split into choice attacker vs doing something else with the choice [usually scarf])
- Setup Sweepers (need to space out these moves, some may be defensive)
- Priority users (may need to split this up, including some maybe in weather, and considering abilities like Technician)
- Stallbreaker (not sure what this will have: usually taunt and setup it seems, and setup may be a separate thing so Taunt may already capture this, maybe perish song as well and other phazing moves, though those are also stalling tactics so not sure, definitely taunt though)

Most of these we can just ignore. Physical and special moves are by and large something we will deal with when we are done investigating roles for features, and we move to creating aggregate features out of our cindex dataframes.

Items are outside the clearly defined scope of this project.

Stallbreaking mainly has to do with setup moves (which is another thing we have to look at anyways) and Taunt (which we already decided will be in our miscellaneous status feature later on).

So, we need to worry about Setup moves and Priority moves

In [ ]:
moves_cindex.loc[['Acid Armor', 'Agility', 'Amnesia', 'Autotomize', 'Belly Drum', 'Bulk Up', 'Calm Mind', 'Clangorous Soul', 'Coil', 'Cosmic Power', 'Cotton Guard', 'Curse', 'Dragon Dance', 'Flame Charge', 'Focus Energy', 'Geomancy', 'Growth', 'Hone Claws', 'Howl', 'Meteor Beam', 'Nasty Plot', 'No Retreat', 'Power-Up Punch', 'Quiver Dance', 'Shell Smash', 'Shift Gear', 'Swords Dance', 'Work Up'], :].sort_values(by='cindex')

In [ ]:
abilities_cindex.loc[['Speed Boost', 'Sniper', 'Guts', 'Unburden', 'Weak Armor'], :].sort_values(by='cindex')

**A possible hierarchy for setup moves:**

1st Level: Shell Smash, Quiver Dance, No Retreat, Geomancy, Clangorous Soul, *Guts*, *Speed Boost*

2nd Level: Swords Dance, Coil, Nasty Plot, Shift Gear, Dragon Dance, *Unburden*

3rd Level: Flame Charge, Belly Drum, Autotomize, Calm Mind, Bulk Up, Cotton Guard, Meteor Beam, Cosmic Power, Growth, Agility, *Sniper*, *Weak Armor*

4th Level: Acid Armor, Work Up, Amnesia, Hone Claws, Curse, Power-Up Punch

5th Level: Howl

*italics: abilities rather than moves*

Most of this is pretty straightforward from the relative tiers of cindexes of moves and abilities. We're not including focus energy because the only pokemon that uses it is Kingdra, and that's because of its ability Sniper which is much better and is already included even higher up in the setup hierarchy. Howl is the lowest since all it does is raise attack one stage; it's cindex is slightly higher than some 4th level moves but it's not actually a better move.

In [ ]:
#Offensive: Setup
shrinkfeatures_df['Setup'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Howl'] == 1, 'Setup'] = 1
moves_cindex.drop(index=['Howl'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Acid Armor'] == 1, 'Setup'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Work Up'] == 1, 'Setup'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Amnesia'] == 1, 'Setup'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Hone Claws'] == 1, 'Setup'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Curse'] == 1, 'Setup'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Power-Up Punch'] == 1, 'Setup'] = 2
moves_cindex.drop(index=['Acid Armor', 'Work Up', 'Amnesia', 'Hone Claws', 'Curse', 'Power-Up Punch'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Flame Charge'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Belly Drum'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Autotomize'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Calm Mind'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Bulk Up'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Cotton Guard'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Meteor Beam'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Cosmic Power'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Growth'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Agility'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_abilities_df['Sniper'] == 1, 'Setup'] = 3
shrinkfeatures_df.loc[pokemon_abilities_df['Weak Armor'] == 1, 'Setup'] = 3
moves_cindex.drop(index=['Flame Charge', 'Belly Drum', 'Autotomize', 'Calm Mind', 'Bulk Up', 'Cotton Guard', 'Meteor Beam', 'Cosmic Power', 'Agility'], inplace=True) #can't remove growth because I'll need it for weather
shrinkfeatures_df.loc[pokemon_learnsets_df['Swords Dance'] == 1, 'Setup'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Coil'] == 1, 'Setup'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Nasty Plot'] == 1, 'Setup'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Shift Gear'] == 1, 'Setup'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Dragon Dance'] == 1, 'Setup'] = 4
shrinkfeatures_df.loc[pokemon_abilities_df['Unburden'] == 1, 'Setup'] = 4
moves_cindex.drop(index=['Swords Dance', 'Coil', 'Nasty Plot', 'Shift Gear', 'Dragon Dance'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Shell Smash'] == 1, 'Setup'] = 5
shrinkfeatures_df.loc[pokemon_learnsets_df['Quiver Dance'] == 1, 'Setup'] = 5
shrinkfeatures_df.loc[pokemon_learnsets_df['No Retreat'] == 1, 'Setup'] = 5
shrinkfeatures_df.loc[pokemon_learnsets_df['Geomancy'] == 1, 'Setup'] = 5
shrinkfeatures_df.loc[pokemon_learnsets_df['Clangorous Soul'] == 1, 'Setup'] = 5
shrinkfeatures_df.loc[pokemon_abilities_df['Guts'] == 1, 'Setup'] = 5
shrinkfeatures_df.loc[pokemon_abilities_df['Speed Boost'] == 1, 'Setup'] = 5
moves_cindex.drop(index=['Shell Smash', 'Quiver Dance', 'No Retreat', 'Geomancy', 'Clangorous Soul'], inplace=True)

shrinkfeatures_df['Setup'].value_counts()

<a id="offense_priority"></a>
### Offensive: Priority

In [ ]:
moves_cindex.loc[['Accelerock', 'Aqua Jet', 'Baby-Doll Eyes', 'Bullet Punch', 'Crafty Shield', 'Detect', 'Endure', 'Extreme Speed', 'Fake Out', 'First Impression', 'Grassy Glide', 'Ice Shard', 'Mach Punch', 'Magic Coat', 'Quick Attack', 'Quick Guard', 'Protect', 'Shadow Sneak', 'Sucker Punch', 'Vacuum Wave', 'Water Shuriken', 'Wide Guard'], :].sort_values(by='cindex')

In [ ]:
abilities_cindex.loc[['Gale Wings', 'Prankster', 'Triage', 'Technician'], :].sort_values(by='cindex')

Although there are a range of cindexes here, some of it is misleading. I think Fake Out and Quick Attack are the least impactful since they are low cindex and normal type, so they will be "1" for this feature.

Then we have the classic 40 powered priority moves: Accelerock, Shadow Sneak, Mach Punch, Aqua Jet, Bullet Punch, Ice Shard and Vacuum Wave. Although they have significant cindex range, they really do very similar things so I can't really justify including them in a separate tier, so they will all be two.

Then we have extreme speed, first impression, and sucker punch. They are more powerful than the other moves but don't have clearly higher cindex and they have limitations as well: extreme speed is the normal type which can never be supereffective, first impression can only be used on the first turn a pokemon is sent out, and sucker punch only works if the opponent attacks with a damaging move, giving the opponent some control over it. So I will also consider these moves as a "2" for this feature.

The abilities are not priority moves themselves, but they take priority moves to another level of effectiveness, thus enabling an entire style around them. They also have heightened average cindex, inevitably come as a package deal with priority moves lower mentioned previously to be lower in the hierarchy, or make moves into priority moves which otherwise weren't (otherwise the pokemon would not have the ability), and the low cindex of Triage is misleading since only one pokemon, Comfey, has it, for some reason its moveset with Triage was not included in the data but I have seen it used very effectively (it just also has Natural Cure, so it is usually used in a Status Absorber role instead, but Triage is a good ability when used, so even if it's worse than natural cure does not actually make it "0" cindex level). Because of all of this, the abilities will be a "3".

The other abilities are either:
- garbage
- defensively oriented and close to garbage and thus unrelated to what is meant with this feature (we aren't  talking about shield moves even though they have priority and most are terrible anyways)
- in so many learnsets that they give no info (e.g. endure, protect).

Grassy Slide is more of a weather oriented feature than a priority move, since it is dependent on terrain.

In [ ]:
#Offensive: Priority
shrinkfeatures_df['Priority'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Fake Out'] == 1, 'Priority'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Quick Attack'] == 1, 'Priority'] = 1
moves_cindex.drop(index=['Fake Out', 'Quick Attack'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Accelerock'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Shadow Sneak'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['First Impression'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Mach Punch'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Aqua Jet'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Bullet Punch'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Ice Shard'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Extreme Speed'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Sucker Punch'] == 1, 'Priority'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Vacuum Wave'] == 1, 'Priority'] = 2
moves_cindex.drop(index=['Accelerock', 'Shadow Sneak', 'First Impression', 'Mach Punch', 'Aqua Jet', 'Bullet Punch', 'Ice Shard', 'Extreme Speed', 'Sucker Punch', 'Vacuum Wave'], inplace=True)
shrinkfeatures_df.loc[pokemon_abilities_df['Gale Wings'] == 1, 'Priority'] = 3
shrinkfeatures_df.loc[pokemon_abilities_df['Prankster'] == 1, 'Priority'] = 3
shrinkfeatures_df.loc[pokemon_abilities_df['Triage'] == 1, 'Priority'] = 3
shrinkfeatures_df.loc[pokemon_abilities_df['Technician'] == 1, 'Priority'] = 3

shrinkfeatures_df['Priority'].value_counts()

<a id="defense_drain"></a>
### Defensive: HP Drain and Recovery

While I won't make a feature specific to defensive pokemon like Tanks or Walls (which is mostly based on stats and/or type), I want to try making a feature based on recovery. This could be split between moves which heal hp directly like Recover, Roost and Strength Sap (even better really), and moves which drain hp like drain punch, giga drain, leech seed, etc. I should probably investigate this recovery feature in more detail (https://bulbapedia.bulbagarden.net/wiki/Category:Moves_that_restore_HP):

In [ ]:
moves_cindex.loc[['Aqua Ring', 'Drain Punch', 'Draining Kiss', 'Dream Eater', 'Giga Drain', 'Horn Leech', 'Ingrain', 'Leech Life', 'Leech Seed', 'Life Dew', 'Moonlight', 'Oblivion Wing', 'Parabolic Charge', 'Purify', 'Recover', 'Rest', 'Roost', 'Shore Up', 'Strength Sap'], :].sort_values(by='cindex')

In [ ]:
abilities_cindex.loc[['Regenerator'], :].sort_values(by='cindex')

Draining Move Hierarchy based on cindexes:
- Top: Oblivion Wing
- Second: Horn Leech
- Third: Giga Drain, Leech Seed, Drain Punch
- Fourth: Leech Life, Draining Kiss

In [ ]:
shrinkfeatures_df['HP Drain'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Leech Life'] == 1, 'HP Drain'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Draining Kiss'] == 1, 'HP Drain'] = 1
moves_cindex.drop(index=['Leech Life', 'Draining Kiss'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Giga Drain'] == 1, 'HP Drain'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Leech Seed'] == 1, 'HP Drain'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Drain Punch'] == 1, 'HP Drain'] = 2
moves_cindex.drop(index=['Giga Drain', 'Leech Seed', 'Drain Punch'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Horn Leech'] == 1, 'HP Drain'] = 3
moves_cindex.drop(index=['Horn Leech'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Oblivion Wing'] == 1, 'HP Drain'] = 4
moves_cindex.drop(index=['Oblivion Wing'], inplace=True)

shrinkfeatures_df['HP Drain'].value_counts()

Recovery Move Hierarchy:
- Top: Shore Up, Strength Sap, *Regenerator*
- Second: Roost, Recover, Moonlight (Moonlight has better recovery in certain weather but lower PP so it balances out)

In [ ]:
#Defensive: HP Recovery
shrinkfeatures_df['HP Recovery'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Roost'] == 1, 'HP Recovery'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Recover'] == 1, 'HP Recovery'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Moonlight'] == 1, 'HP Recovery'] = 1
moves_cindex.drop(index=['Roost', 'Recover', 'Moonlight'], inplace=True)
shrinkfeatures_df.loc[pokemon_learnsets_df['Shore Up'] == 1, 'HP Recovery'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Strength Sap'] == 1, 'HP Recovery'] = 2
shrinkfeatures_df.loc[pokemon_abilities_df['Regenerator'] == 1, 'HP Recovery'] = 2
moves_cindex.drop(index=['Shore Up', 'Strength Sap'], inplace=True)

shrinkfeatures_df['HP Recovery'].value_counts()

<a id="weather"></a>
### Weather

- Weather/Terrain Setters
- Weather/Terrain Speedsters
- Weather Advantage? (Sand Veil, Aurora Veil even though its a move, SolarBeam/Thunder, other abusers?, maybe make this into a stat that makes speeders a 2 and pokemon with weather advantage moves/type as a 1, or maybe even up to 3 taking into account type, moveset and speed gain and maybe other advantages as well, maybe a tiered weather abuse feature), moves and abilities
- Trick Room

In [ ]:
abilities_cindex.loc[['Drought', 'Drizzle', 'Sand Stream', 'Snow Warning', 'Grassy Surge', 'Misty Surge', 'Psychic Surge', 'Electric Surge'], :].sort_values(by='cindex')

These are all fantastic and very important features and all have 100 cindex so they can just be one-hot encoded into a "weather set" feature.

There are a few kinds of abilities and moves which benefit from weather and terrain, but the one's that stand out in competitive are abilities which double speed in certain weather or terrain, so those should probably be made something separate from the others as they are the most important to including (especially on weather teams)

In [ ]:
abilities_cindex.loc[['Chlorophyll', 'Swift Swim', 'Sand Rush', 'Slush Rush', 'Surge Surfer'], :].sort_values(by='cindex')

Slush Rush/Surge Surfer only seem to have higher cindex due to the lower access to the ability, but all of them are very good and deserve to be at or near the top of a weather gimmick feature, and if anything the 100 cindex one's are used less in competitive strategies since their version of weather is rarer

Now we need to consider other abilities which take advantage of weather or terrain and consider whether they are worthwhile to include in a feature (as well as moves):

In [ ]:
abilities_cindex.loc[['Dry Skin', 'Flower Gift', 'Leaf Guard', 'Solar Power', 'Hydration', 'Rain Dish', 'Sand Veil', 'Sand Force', 'Ice Body', 'Snow Cloak', 'Ice Face', 'Mimicry'], :].sort_values(by='cindex')

In [ ]:
moves_cindex.loc[['Thunder', 'Hurricane', 'Blizzard', 'Solar Beam', 'Solar Blade', 'Growth', 'Aurora Veil', 'Expanding Force', 'Grassy Glide', 'Misty Explosion', 'Rising Voltage', 'Terrain Pulse', 'Weather Ball', 'Nature Power'], :].sort_values(by='cindex')

Some like Hurricane seem pretty good, but in general these moves aren't very good so I feel like it's hard to justify making a feature here, it seems like it would be misleading or not predictive to make a feature (for taking advantage of weather) including something like Thunder that very few pokemon use but many have access to. I will include it into the aggregate weather gimmick feature if it's good.

I had one more feature, which was Trick Room.  I should probably investigate it more to make sure there isn't anything to the strategy besides using the move and using naturally slow pokemon

In [ ]:
roles_dict['Trick Room']

In [ ]:
moves_cindex.loc[['Trick Room'], :].sort_values(by='cindex')

It's a subpar move and doesn't seem to have any other systematic moves or abilities used with it, plus in every pokemon that uses it, better strategies are almost always listed. I may aggregate it into misc status later but it doesn't deserve to be included as a strong weather feature and it isn't really weather or terrain so fits in a rather questionable way with it.

Hierarchy of these Weather/Terrain abilities:
- Top: the weather speed moves
- Second: Mimicry, Ice Face, Dry Skin
- Third: Solar Power, Hurricane
- Fourth: Snow Cloak, Sand Force, Leaf Guard, Aurora Veil, Misty Explosion, Grassy Slide, Growth, Weather Ball, Thunder, Rising Voltage, Solar Blade
- Fifth: Ice Body, Sand Veil, Rain Dish, Hydration, Flower Gift, Expanding Force, Blizzard, Solar Beam, Nature Power, Terrain Pulse

In [ ]:
#Weather Set
shrinkfeatures_df['Weather Set'] = pokemon_abilities_df.loc[:, ['Drought', 'Drizzle', 'Sand Stream', 'Snow Warning', 'Grassy Surge', 'Misty Surge', 'Psychic Surge', 'Electric Surge']].sum(axis=1)

shrinkfeatures_df['Weather Set'].value_counts()

In [ ]:
#Weather Gimmick (only remove moves if you'd only use them in weather)
shrinkfeatures_df['Weather Speed'] = pokemon_abilities_df.loc[:, ['Chlorophyll', 'Swift Swim', 'Sand Rush', 'Slush Rush', 'Surge Surfer']].sum(axis=1)
shrinkfeatures_df.loc[:, 'Weather Speed'].where(shrinkfeatures_df['Weather Speed'] <= 1, 1, inplace=True)
shrinkfeatures_df['Weather Gimmick'] = 0
shrinkfeatures_df.loc[pokemon_abilities_df['Ice Body'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_abilities_df['Sand Veil'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_abilities_df['Rain Dish'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_abilities_df['Hydration'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_abilities_df['Flower Gift'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Expanding Force'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Blizzard'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Solar Beam'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Nature Power'] == 1, 'Weather Gimmick'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Terrain Pulse'] == 1, 'Weather Gimmick'] = 1
moves_cindex.drop(index=['Expanding Force', 'Solar Beam', 'Nature Power', 'Terrain Pulse'], inplace=True) #won't remove Blizzard
shrinkfeatures_df.loc[pokemon_abilities_df['Snow Cloak'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_abilities_df['Sand Force'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_abilities_df['Leaf Guard'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Aurora Veil'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Misty Explosion'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Grassy Glide'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Growth'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Weather Ball'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Thunder'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Rising Voltage'] == 1, 'Weather Gimmick'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Solar Blade'] == 1, 'Weather Gimmick'] = 2
moves_cindex.drop(index=['Aurora Veil', 'Misty Explosion', 'Grassy Glide', 'Growth', 'Weather Ball', 'Rising Voltage', 'Solar Blade'], inplace=True) #won't remove Thunder
shrinkfeatures_df.loc[pokemon_abilities_df['Solar Power'] == 1, 'Weather Gimmick'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Hurricane'] == 1, 'Weather Gimmick'] = 3 #won't remove Hurricane
shrinkfeatures_df.loc[pokemon_abilities_df['Mimicry'] == 1, 'Weather Gimmick'] = 4
shrinkfeatures_df.loc[pokemon_abilities_df['Ice Face'] == 1, 'Weather Gimmick'] = 4
shrinkfeatures_df.loc[pokemon_abilities_df['Dry Skin'] == 1, 'Weather Gimmick'] = 4
#add weather speed to weather gimmick and drop weather speed
shrinkfeatures_df.loc[shrinkfeatures_df['Weather Speed'] == 1, 'Weather Gimmick'] = 5
shrinkfeatures_df.drop(columns='Weather Speed', inplace=True)

shrinkfeatures_df['Weather Gimmick'].value_counts()

<a id="indiscriminate"></a>
### Indiscriminate Moves

In [ ]:
moves_cindex.loc[moves_cindex['category'] == 'Non-Damaging'].sort_values(by='n_learnset', ascending=False).head(15)

We need to remove all the moves above Reflect on this list which have above 0 cindex (the 0 cindex one's will never make it into an aggregate feature anyways so they are irrelevant), because they are used by too many pokemon to have any discriminatory power (more than half), regardless of what their cindex is (such as Toxic's being very high). We are also going to include Focus Energy since its usage is completed supplanted by the ability Sniper which is better than the move and the only reason you would ever use it

In [ ]:
#Indiscriminate moves (give no information due to overly common access)
moves_cindex.drop(index=['Substitute', 'Protect', 'Rest', 'Sleep Talk', 'Sunny Day', 'Rain Dance', 'Confide', 'Attract', 'Endure', 'Focus Energy', 'Toxic'], inplace=True)

The general reason I have been removing moves from moves_cindex but not removing abilities from abilities_cindex is because it leaves some pokemon without abilities (abilities are far more scarce than moves since each pokemon only has access to 4 or fewer abilities and it's almost always fewer), and it severely misleads about the strength of abilities for the pokemon since they will have their strongest abilities taken away in the aggregate ability feature (for moves having a few moves taken away will not be relevant, there are 100s of moves for most pokemon and we are measuring mostly different kinds of moves than are being used in features, focusing on offensive moves in the aggregate moves features, abilities have no such divisions).

<a id="moves_cindex_invest"></a>
### Investigating moves_cindex

With all of the features we created and all the final adjustments to moves_cindex made on that basis, we can quantitatively inspect moves_cindex with the aim of attaining final insights from it and aggregating new quantitative features for the remaining moves, which is still quite a lot:

In [ ]:
moves_cindex.shape

In [ ]:
moves_cindex['cindex'].describe(percentiles = [n/20 for n in range(1, 20)])

Now close to 55% of our moves have a 0 cindex!  That's a lot to remove, because 0 cindex moves don't tell us anything meaningful about the distribution of moves since they are not competitively viable. The mean is about 15.  There are plenty of moves with 100 cindex that we need to investigate.

In [ ]:
moves_cindex.loc[moves_cindex['cindex'] == 100]

These all have an n_learnset of 1 or 2, except Multi-Attack which is the special move of Silvally (thus in a sense its n_learnset is only 1 since Silvally just has many versions, but since all versions use it then it's probably a very good move). These moves will be put into a "Unique Powerful Move" feature later which will be one-hot encoded, but let's understand more about the distribution of moves_cindex.

In [ ]:
moves_cindex.loc[moves_cindex['n_learnset'] == 1, 'cindex'].describe(percentiles = [n/20 for n in range(1, 20)])

These unique moves skew the cindex distribution a lot because of how many of them there are (77) and because of their inherently extreme cindex values

In [ ]:
moves_cindex[moves_cindex['n_learnset'] == 2].sort_values(by='cindex')

These n_learnset=2 moves also have extreme values most of the time.

In [ ]:
moves_cindex[moves_cindex['n_learnset'] == 3].sort_values(by='cindex')

Whether these values are extreme is more questionable.

These distributions are still quite uninteresting, so it may be worth excluding from the distribution moves with n_learnset of 2 and 3 as well

- Question: What do we do with moves with n_learnset 2 or 3, but which aren't 0 or 100? Are they included in the pokemon unique move feature?

Let's look at the value_counts of n_learnset to get an understanding of what a good cutoff point might be for our unique moves and for what to include in the cindex distribution (versus what is part of an outlier):

In [ ]:
n_learnset = moves_cindex['n_learnset'].value_counts()
n_learnset[n_learnset > 3]

It looks like only getting rid of moves with an n_learnset value of 1 and 2 from the distribution would be the most natural cutoff point, since the other values of n_learnset stop following a sequential pattern and certainly are less extreme.

We can easily assess this with z scores:

In [ ]:
n_learnset[np.abs(stats.zscore(n_learnset)) > 3]

In [ ]:
n_learnset[np.abs(stats.zscore(n_learnset)) > 2]

As 1 and 2 are the only values of n_learnset with z scores higher than both 2 AND 3, there can be no question that they are the only outliers. This gives us good justification to remove them from the distribution of moves_cindex, and treat their 100 cindex moves differently (as they are also the only values that have 100 cindex moves for the most part) by including them in a unique one-hot encoded feature.

However, since n_learnset=2 still has some cindex values of 50, I will still allow it to be included when determining aggregate features for the number of moves a pokemon has above certain thresholds (the distribution for moves above cutoff points is less important, the raw count is more important, so the extreme values held by moves with such n_learnset are mostly unimportant, with the exception of the 100 cindex moves since those are part of the special features thus those will just be removed anyways)

Let's remove the n_learnset=(1 or 2) features for now and take a look at the distribution:

In [ ]:
moves_cindex_distro = moves_cindex[moves_cindex['n_learnset'] > 2].copy()
moves_cindex_distro

Let's not forget to drop Multi-Attack from this as it's essentially a unique move

In [ ]:
moves_cindex_distro.drop(index='Multi-Attack', inplace=True)
moves_cindex_distro.shape

Let's look at the distribution one more time:

In [ ]:
moves_cindex_distro.describe(percentiles = [n/20 for n in range(1, 20)])

Some interesting values occur here for all of our columns:

- the median value for n_learnset is about 40 pokemon, the first quartile is about 17 pokemon, and the third quartile is about 75 pokemon. Since the max is so high (nearly all pokemon used for this exercise, which was only 415), it shifts the mean (up to about 55) in such a way as to make it much less useful than the median
- n_competitive tells us that only about half of the moves are even used competitively, and of those, about half are included in 3 or fewer competitive movesets, moves that are used in 11 or more movesets are already in the top 20% of competitive moves and moves are used in at most 72 movesets (which must be a very powerful move)

We won't use those two features since they were constructed arbitrarily based on pokemon that have competitive movesets, which has only marginally to do with their base stats that the model will actually be using to rate them, but nonetheless they are interesting to know

- It's still the case that about 50% of moves have a nonzero cindex (i.e. are used competitively), though it will be more interesting to view the distribution of nonzero values more closely, via describe and hist
- About 30% of moves have zero power (i.e. are non-damaging or "status" moves), so it may also be interesting to view the distribution of nonzero power moves via describe and hist (since we will use a power feature of moves of how many powerful moves a pokemon has of many types to get an idea of its physical and special coverage)
- I also briefly tried kernel density estimation during experiments, but it wasn't useful so I'm not even going to link it, feel free to look for it in the experiments notebook if it interests you enough

In [ ]:
cindex_distro = moves_cindex_distro.loc[moves_cindex_distro['cindex'] > 0, 'cindex']
cindex_distro.describe(percentiles = [n/20 for n in range(1, 20)])

Here we have the cindex distro for competitive moves (moves with above 0 cindex), which have a median of about 11, mean of about 17, and a top 5% with about 49 as a cutoff point

In [ ]:
cindex_distro[cindex_distro >= 48.981481]

These are the top 5% of competitive moves, a nice mix of physical, special and status moves

In [ ]:
cindex_distro.plot.hist(by='cindex')
plt.show()

This actually resembles a pretty smooth exponential curve, which means it's probably a good idea to take the log of it to normalize it.

In [ ]:
cindex_distro_log = cindex_distro.apply(np.log)
cindex_distro_log.plot.hist(by='cindex', bins=20)
plt.show()

To look at where the cutoffs of these bins are more closely:

In [ ]:
pd.cut(cindex_distro_log, 20).value_counts().sort_index()

Some different optional cutoff points that occur for an aggregate feature:
- -0.5, by 6 (being competitive at all, could just use >0 without the log)
- -0.0105 by 3
- 0.232 by 1
- 0.474 by 1
- 0.716 by 2
- 1.2 by 6
- 1.684 by 2
- 2.168 by 10
- 2.653 by 11
- 4.105 by 1

That said, I probably won't use those cutoffs since I need to split moves into physical, special and status first, but it gives the basic model for one of my aggregate features.

In [ ]:
power_distro = moves_cindex_distro.loc[moves_cindex_distro['power'] > 0, 'power']
power_distro.describe(percentiles = [n/20 for n in range(1, 20)])

Nonzero powered moves have a median power of 75, a mean power of about 75 (which is interesting, probably indicating something less exponential and skewed), and a top 10% power of 120.

In [ ]:
power_distro.plot.hist(by='power')
plt.show()

This is a very different distribution from the cindex, not exponential. It's closer to normal, but significantly right tailed. So perhaps our cutoffs would be slightly more concentrated on the left side, but no need to take the log of them since they aren't exponentially distributed.

In [ ]:
power_distro.plot.hist(by='power', bins=20)
plt.show()

In [ ]:
#to see the specific values
pd.cut(power_distro, 20).value_counts().sort_index()

Going by the histogram cuts and fine visualization, some decent cutoffs seem to be (i.e. the "jumps" upward):
- 40 or 38.5
- 75 or 73.75
- 110 or 109.0
- 145 or 144.25

These capture the largest jumps which may capture key checkpoints in move strength.

<a id="physical_cutoff"></a>
### Aggregate Feature: Physical Cutoff

This feature will list the number of moves a pokemon has which are above a certain cindex value. We'll investigate which cutoff is best later.

First we need to consider only the moves in the distribution which are physical in nature to make decisions about this aggregate feature.

In [ ]:
moves_cindex_distro_phys = moves_cindex_distro.loc[moves_cindex_distro['category'] == 'Physical', :].copy()
moves_cindex_distro_phys.drop(columns='category', inplace=True)
moves_cindex_distro_phys.describe(percentiles = [n/20 for n in range(1, 20)])

Physical Moves Summary Statistics:
- n_learnset mean: 58
- n_learnset median: 43
- n_learnset max: 411

- n_competitive mean: 3.5
- n_competitive median: 1
- n_competitive max: 72

- cindex mean: 8
- cindex median: 0.7
- cindex max: 77

- power mean: 66
- power median: 70
- power max: 250

Now we need the physical moves with cindex greater than 0

In [ ]:
physical_distro = moves_cindex_distro_phys.loc[moves_cindex_distro_phys['cindex'] > 0, 'cindex']
physical_distro.describe(percentiles = [n/20 for n in range(1, 20)])

In [ ]:
physical_distro.plot.hist(by='cindex')
plt.show()

It still vaguely resembles exponential decay, but the sparsity starts around 25.  So we can still try a log:

In [ ]:
physical_distro_log = physical_distro.apply(np.log)
physical_distro_log.describe(percentiles = [n/20 for n in range(1, 20)])

In [ ]:
physical_distro_log.plot.hist(bins=20)
plt.show()

In [ ]:
pd.cut(physical_distro_log, 20).value_counts().sort_index()

Cutoff options:
- all of them (how many physical moves a pokemon has)
- 0.474
- 1.442
- 2.168
- 2.653
- 3.379

We will make 6 aggregate physical move features based on these cutoffs

In [ ]:
physical_cutoff1 = list(physical_distro_log.index)
shrinkfeatures_df['Physical Cutoff 1'] = pokemon_learnsets_df.loc[:, physical_cutoff1].sum(axis=1)
shrinkfeatures_df['Physical Cutoff 1'].sort_values(ascending=False).head(10)

In [ ]:
physical_cutoff2 = list(physical_distro_log[physical_distro_log >= 0.474].index)
shrinkfeatures_df['Physical Cutoff 2'] = pokemon_learnsets_df.loc[:, physical_cutoff2].sum(axis=1)
shrinkfeatures_df['Physical Cutoff 2'].sort_values(ascending=False).head(10)

In [ ]:
physical_cutoff3 = list(physical_distro_log[physical_distro_log >= 1.442].index)
shrinkfeatures_df['Physical Cutoff 3'] = pokemon_learnsets_df.loc[:, physical_cutoff3].sum(axis=1)
shrinkfeatures_df['Physical Cutoff 3'].sort_values(ascending=False).head(10)

In [ ]:
physical_cutoff4 = list(physical_distro_log[physical_distro_log >= 2.168].index)
shrinkfeatures_df['Physical Cutoff 4'] = pokemon_learnsets_df.loc[:, physical_cutoff4].sum(axis=1)
shrinkfeatures_df['Physical Cutoff 4'].sort_values(ascending=False).head(10)

In [ ]:
physical_cutoff5 = list(physical_distro_log[physical_distro_log >= 2.653].index)
shrinkfeatures_df['Physical Cutoff 5'] = pokemon_learnsets_df.loc[:, physical_cutoff5].sum(axis=1)
shrinkfeatures_df['Physical Cutoff 5'].sort_values(ascending=False).head(10)

In [ ]:
physical_cutoff6 = list(physical_distro_log[physical_distro_log >= 3.379].index)
shrinkfeatures_df['Physical Cutoff 6'] = pokemon_learnsets_df.loc[:, physical_cutoff6].sum(axis=1)
shrinkfeatures_df['Physical Cutoff 6'].sort_values(ascending=False).head(10)

<a id="physical_coverage"></a>
### Aggregate Feature: Physical Coverage

In [ ]:
physical_powerdistro = moves_cindex_distro_phys.loc[moves_cindex_distro_phys['power'] > 0, ['power', 'type']]
physical_powerdistro['power'].plot.hist(bins=20)
plt.show()

In [ ]:
pd.cut(physical_powerdistro['power'], 20).value_counts().sort_index()

Cutoff options:
- all of them
- 38.5
- 73.75
- 109

We'll make an aggregate feature for these four cutoffs of how many types that a pokemon has available moves for which are also above those levels of power (which gives an idea of how many types a pokemon can "cover" offensively, i.e. how versatile it is offensively against pokemon of a variety of types), and we'll define a function to help us do this

In [ ]:
def physical_coveragesum(cutoff):
    powerdistro = physical_powerdistro.loc[physical_powerdistro['power'] >= cutoff, :]
    coveragesum = pd.DataFrame(index=shrinkfeatures_df.index)
    for ptype in type_list:
        physical_coverage = list(powerdistro.loc[powerdistro['type'] == ptype, :].index)
        coveragesum[ptype] = pokemon_learnsets_df.loc[:, physical_coverage].sum(axis=1)
        coveragesum.loc[:, ptype].where(coveragesum[ptype] <= 1, 1, inplace=True)
    return coveragesum.sum(axis=1)

In [ ]:
shrinkfeatures_df['Physical Coverage 1'] = physical_coveragesum(0)
shrinkfeatures_df['Physical Coverage 1'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Physical Coverage 2'] = physical_coveragesum(38.5)
shrinkfeatures_df['Physical Coverage 2'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Physical Coverage 3'] = physical_coveragesum(73.75)
shrinkfeatures_df['Physical Coverage 3'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Physical Coverage 4'] = physical_coveragesum(109)
shrinkfeatures_df['Physical Coverage 4'].sort_values(ascending=False).head(10)

<a id="special_cutoff"></a>
### Aggregate Feature: Special Cutoff

Now we will make a precisely analogous feature for special moves as we did in the feature physical cutoff.

In [ ]:
moves_cindex_distro_spec = moves_cindex_distro.loc[moves_cindex_distro['category'] == 'Special', :].copy()
moves_cindex_distro_spec.drop(columns='category', inplace=True)
moves_cindex_distro_spec.describe(percentiles = [n/20 for n in range(1, 20)])

Special Moves Summary Statistics:
- n_learnset mean: 60
- n_learnset median: 38
- n_learnset max: 413
- n_competitive mean: 5.5
- n_competitive median: 1
- n_competitive max: 37
- cindex mean: 10
- cindex median: 2
- cindex max: 72
- power mean: 74
- power median: 75
- power max: 160

Now we need the special moves with cindex greater than 0

In [ ]:
special_distro = moves_cindex_distro_spec.loc[moves_cindex_distro_spec['cindex'] > 0, 'cindex']
special_distro.describe(percentiles = [n/20 for n in range(1, 20)])

In [ ]:
special_distro.plot.hist(by='cindex')
plt.show()

It's close enough to the physical moves that we can still try using a log transform

In [ ]:
special_distro_log = special_distro.apply(np.log)
special_distro_log.describe(percentiles = [n/20 for n in range(1, 20)])

In [ ]:
special_distro_log.plot.hist(bins=20)
plt.show()

In [ ]:
pd.cut(special_distro_log, 20).value_counts().sort_index()

Cutoff options:
- all of them
- 0.607
- 1.219
- 1.627
- 2.239
- 2.647
- 3.055

We will make 7 aggregate special move features based on these cutoffs

In [ ]:
special_cutoff1 = list(special_distro_log.index)
shrinkfeatures_df['Special Cutoff 1'] = pokemon_learnsets_df.loc[:, special_cutoff1].sum(axis=1)
shrinkfeatures_df['Special Cutoff 1'].sort_values(ascending=False).head(10)

In [ ]:
special_cutoff2 = list(special_distro_log[special_distro_log >= 0.607].index)
shrinkfeatures_df['Special Cutoff 2'] = pokemon_learnsets_df.loc[:, special_cutoff2].sum(axis=1)
shrinkfeatures_df['Special Cutoff 2'].sort_values(ascending=False).head(10)

In [ ]:
special_cutoff3 = list(special_distro_log[special_distro_log >= 1.219].index)
shrinkfeatures_df['Special Cutoff 3'] = pokemon_learnsets_df.loc[:, special_cutoff3].sum(axis=1)
shrinkfeatures_df['Special Cutoff 3'].sort_values(ascending=False).head(10)

In [ ]:
special_cutoff4 = list(special_distro_log[special_distro_log >= 1.627].index)
shrinkfeatures_df['Special Cutoff 4'] = pokemon_learnsets_df.loc[:, special_cutoff4].sum(axis=1)
shrinkfeatures_df['Special Cutoff 4'].sort_values(ascending=False).head(10)

In [ ]:
special_cutoff5 = list(special_distro_log[special_distro_log >= 2.239].index)
shrinkfeatures_df['Special Cutoff 5'] = pokemon_learnsets_df.loc[:, special_cutoff5].sum(axis=1)
shrinkfeatures_df['Special Cutoff 5'].sort_values(ascending=False).head(10)

In [ ]:
special_cutoff6 = list(special_distro_log[special_distro_log >= 2.647].index)
shrinkfeatures_df['Special Cutoff 6'] = pokemon_learnsets_df.loc[:, special_cutoff6].sum(axis=1)
shrinkfeatures_df['Special Cutoff 6'].sort_values(ascending=False).head(10)

In [ ]:
special_cutoff7 = list(special_distro_log[special_distro_log >= 3.055].index)
shrinkfeatures_df['Special Cutoff 7'] = pokemon_learnsets_df.loc[:, special_cutoff7].sum(axis=1)
shrinkfeatures_df['Special Cutoff 7'].sort_values(ascending=False).head(10)

<a id="special_coverage"></a>
### Aggregate Feature: Special Coverage

Finally we'll make another analogous set of features for special coverage, just as we did for physical coverage, but for special moves instead of physical moves.

In [ ]:
special_powerdistro = moves_cindex_distro_spec.loc[moves_cindex_distro_spec['power'] > 0, ['power', 'type']]
special_powerdistro['power'].plot.hist(bins=20)
plt.show()

In [ ]:
pd.cut(special_powerdistro['power'], 20).value_counts().sort_index()

Cutoff options:
- all of them
- 34.0
- 48.0
- 55.0
- 76.0
- 97.0
- 104.0
- 118.0
- 139.0
- 146.0

In [ ]:
def special_coveragesum(cutoff):
    powerdistro = special_powerdistro.loc[special_powerdistro['power'] >= cutoff, :]
    coveragesum = pd.DataFrame(index=shrinkfeatures_df.index)
    for ptype in type_list:
        special_coverage = list(powerdistro.loc[powerdistro['type'] == ptype, :].index)
        coveragesum[ptype] = pokemon_learnsets_df.loc[:, special_coverage].sum(axis=1)
        coveragesum.loc[:, ptype].where(coveragesum[ptype] <= 1, 1, inplace=True)
    return coveragesum.sum(axis=1)

In [ ]:
shrinkfeatures_df['Special Coverage 1'] = special_coveragesum(0)
shrinkfeatures_df['Special Coverage 1'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 2'] = special_coveragesum(34)
shrinkfeatures_df['Special Coverage 2'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 3'] = special_coveragesum(48)
shrinkfeatures_df['Special Coverage 3'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 4'] = special_coveragesum(55)
shrinkfeatures_df['Special Coverage 4'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 5'] = special_coveragesum(76)
shrinkfeatures_df['Special Coverage 5'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 6'] = special_coveragesum(97)
shrinkfeatures_df['Special Coverage 6'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 7'] = special_coveragesum(104)
shrinkfeatures_df['Special Coverage 7'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 8'] = special_coveragesum(118)
shrinkfeatures_df['Special Coverage 8'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 9'] = special_coveragesum(139)
shrinkfeatures_df['Special Coverage 9'].sort_values(ascending=False).head(10)

In [ ]:
shrinkfeatures_df['Special Coverage 10'] = special_coveragesum(146)
shrinkfeatures_df['Special Coverage 10'].sort_values(ascending=False).head(10)

<a id="misc_status"></a>
### Aggregate Feature: Miscellaneous Status

Now we need to make an aggregate feature for the last category of moves, status moves. Since these moves don't do direct damage, we can't use them to make a coverage feature, but we'll still probably want to know their cindexes and the distribution thereof. This feature is going to have more similarity to the manual features we created than to the other aggregate features, since having status moves itself is often a unique utility role that is played (rather than just having generically strong physical or special attacks).

In [ ]:
moves_cindex_distro_stat = moves_cindex_distro.loc[moves_cindex_distro['category'] == 'Non-Damaging', :].copy()
moves_cindex_distro_stat.drop(columns='category', inplace=True)
moves_cindex_distro_stat.describe(percentiles = [n/20 for n in range(1, 20)])

It looks like most status moves are not really significant, so we need not concern ourselves with most of these statistics, and we are only interested in the top 25% or so of status moves.

In [ ]:
moves_cindex_distro_stat.loc[moves_cindex_distro_stat['cindex'] > 0, :].sort_values(by='cindex')

There is one more thing that we are forgetting about, which is that there are several status moves left in our pool of unique moves. Since we'll make a separate feature for unique powerful attacks soon, let's remove the remaining powerful status moves there (which haven't been removed already into something like setup, e.g. Geomancy) and include them in the hierarchy of our misc status feature:

In [ ]:
moves_cindex.loc[(moves_cindex['cindex'] == 100) & (moves_cindex['category'] == 'Non-Damaging'), :]

Lovely Kiss is actually no better than Sleep Powder in terms of its move statistics, but it only has 100 cindex since it is only in the (competitive) learnset of Jynx and it is a good move. Spore, on the other hand, is unique as a 100 accuracy immediate sleep move, so it really is a cut above.

Also, although Thunder Wave has a slightly lower cindex than Stun Spore, it is  an almost strictly better move due to higher accuracy for the same effect. Stun Spore may just have higher cindex due to having much narrow access for a certain group of pokemon that could use it well, whereas Thunder Wave is a widely accessible move, but Thunder Wave will be placed above Stun Spore.

As another reminder, there were also three damaging moves with powerful secondary status effects that cause them to be used for phazing, like many of the moves in our 2nd to last rank of this feature (e.g. Encore, Whirlwind, etc.). These are Clear Smog, Circle Throw and Dragon Tail, and since these moves are used more for their secondary status effects than for their damage, they fit well in the same tier of this feature as the other phazing moves.

Misc Status Hierarchy (based on the most reasonable clusters of above cindexes):
- 5: Spore
- 4: Unique Shield, Sleep Powder, Lovely Kiss, Glare
- 3: Thunder Wave, Trick, Will-O-Wisp
- 2: Destiny Bond, Encore, Memento, Whirlwind, Haze, Pain Split, Yawn, Taunt, Stun Spore, Clear Smog, Dragon Tail, Circle Throw
- 1: Safeguard, Block, Spite, Skill Swap, Magnet Rise, Hail, Magic Coat, Tailwind, Reflect, Copycat, Light Screen, Misty Terrain, Disable, Roar, Trick Room, Perish Song, Soak, Hypnosis

In [ ]:
#Misc Status
shrinkfeatures_df['Misc Status'] = 0
shrinkfeatures_df.loc[pokemon_learnsets_df['Safeguard'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Block'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Spite'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Skill Swap'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Magnet Rise'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Hail'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Magic Coat'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Copycat'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Misty Terrain'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Disable'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Soak'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Tailwind'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Reflect'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Light Screen'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Trick Room'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Roar'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Perish Song'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Hypnosis'] == 1, 'Misc Status'] = 1
shrinkfeatures_df.loc[pokemon_learnsets_df['Memento'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Pain Split'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Taunt'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Whirlwind'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Yawn'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Haze'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Clear Smog'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Destiny Bond'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Encore'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Dragon Tail'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Circle Throw'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Stun Spore'] == 1, 'Misc Status'] = 2
shrinkfeatures_df.loc[pokemon_learnsets_df['Trick'] == 1, 'Misc Status'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Thunder Wave'] == 1, 'Misc Status'] = 3
shrinkfeatures_df.loc[pokemon_learnsets_df['Will-O-Wisp'] == 1, 'Misc Status'] = 3
#don't forget Unique Shield moves, which have to be added separately
#since they are not really duplicate moves, just have similar function
#collectively their cindex would be 75
shrinkfeatures_df.loc[pokemon_learnsets_df['King\'s Shield'] == 1, 'Misc Status'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Spiky Shield'] == 1, 'Misc Status'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Obstruct'] == 1, 'Misc Status'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Baneful Bunker'] == 1, 'Misc Status'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Sleep Powder'] == 1, 'Misc Status'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Lovely Kiss'] == 1, 'Misc Status'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Glare'] == 1, 'Misc Status'] = 4
shrinkfeatures_df.loc[pokemon_learnsets_df['Spore'] == 1, 'Misc Status'] = 5
shrinkfeatures_df['Misc Status'].value_counts()

And that should take care of any status moves that are important competitively, according to the statistics we got from describe and our peripheral knowledge of edge cases.

<a id="unique_powerful"></a>
### Unique Powerful Moves Feature

Now we need to make a feature indicating whether a pokemon has one of the unique powerful moves with cindex 100 and n_learnset 1 or 2 (one has n_learnset 11 but that's a fluke as I already explained about Silvally and Multi-Attack). We should look at all of those moves to make sure we aren't missing anything, but first we need to remember to remove Spore and Lovely Kiss, since this feature is really capturing unique damaging moves (not status moves which our misc status and other many manual features accounted for).

In [ ]:
moves_cindex.drop(index=['Spore', 'Lovely Kiss'], inplace=True)
moves_cindex.loc[moves_cindex['cindex'] == 100, :]

These moves all make sense for the feature, so now we just need to create it:

In [ ]:
shrinkfeatures_df['Unique Powerful Move'] = pokemon_learnsets_df.loc[:, list(moves_cindex.loc[moves_cindex['cindex'] == 100, :].index)].sum(axis=1)
#we're just one-hot encoding this so let's get rid of all values over 1
shrinkfeatures_df.loc[:, 'Unique Powerful Move'].where(shrinkfeatures_df['Unique Powerful Move'] <= 1, 1, inplace=True)
shrinkfeatures_df['Unique Powerful Move'].value_counts()

<a id="abilities_invest"></a>
### Investigating abilities_cindex

Just as we did for moves, we need to make an aggregate feature for the remaining abilities that weren't manually placed into features. In order to make appropriate decisions on how to perform various cutoffs in our aggregation, we need to quantitatively investigate our abilities_cindex dataframe first.

In [ ]:
abilities_cindex['cindex'].describe(percentiles = [n/20 for n in range(1, 20)])

Relative to moves, there is a very large percentage of 100 cindex abilities. We should probably investigate just those 100 cindex abilities more closely:

In [ ]:
abilities_cindex.loc[abilities_cindex['cindex'] == 100].describe(percentiles = [n/20 for n in range(1, 20)])

This is a much more lively distribution than the 100 cindex moves! Although more than half of these abilities are powerful unique abilities (with n_abilityset of only 1), plenty have 2 or 3 for n_abilityset, and there is one with as many as 13! That must be an extremely powerful ability to be used so consistently, given the option to use it.

To make the decision of which n_abilityset we want to use as the cutoff for our distribution and our eventual unique powerful ability feature, we'll have to do some statistical tests. Before we do that though, let's look at the perfect cindex abilities which have n_abilityset higher than 1, and check the distribution of cindex 0 abilities:

In [ ]:
abilities_cindex.loc[(abilities_cindex['cindex'] == 100) & (abilities_cindex['n_abilityset'] > 1)].sort_values(by='n_abilityset')

- Regenerator is a very strong and useful ability that isn't unique to a family or line of pokemon.  Other abilities on this list which fit this description: Natural Cure, Guts, Magic Guard, Scrappy, Adaptability, Speed Boost, Magic Bounce, Drought, Multiscale, Aftermath, Dry Skin, Snow Warning, Sand Stream, Tough Claws, Drizzle, Grassy Surge, Psychic Surge, Magnet Pull, Sticky Hold, Shed Skin, Huge Power, Compound Eyes, Skill Link, Storm Drain, Aroma Veil, Berserk, Electric Surge, Fluffy, Misty Surge, Slush Rush, Trace
- RKS System belongs uniquely to Silvally who just has many variants so that may be misleading (same situation as Multi-Attack)
- Beast Boost belongs to the Ultra Beasts and is very strong and non-negotiable (maybe it should just be considered a unique given the lack of choice, not sure though)
- Unseen Fist belongs to the two forms of Urshifu and is the only option so should be considered unique
- Neutralizing Gas belongs to the Weezing family (including Galar) so should be considered unique
- Dauntless Shield is unique to the versions of Zazamenta and is obligatory so should be considered unique

Statistics will be necessary to make final decisions about what we should do. So just as we made a series called n_learnset for moves that we used to examine z_scores, we should make an analogous series for abilities called n_abilityset.

In [ ]:
n_abilityset = abilities_cindex['n_abilityset'].value_counts()
n_abilityset

In [ ]:
n_abilityset[np.abs(stats.zscore(n_abilityset)) > 3]

In [ ]:
n_abilityset[np.abs(stats.zscore(n_abilityset)) > 2]

So for abilities, n_abilityset=1 is definitively the only outlier value for n_abilityset.  I already looked at the 100 cindex abilities for which n_abilityset > 1, so I already have a good idea of which of those abilities need to be treated uniquely.

I will have to remove n_abilityset=1 abilities from the distribution, since they skew it unreasonably, and remove the 0 values (the uncompetitive abilities), just as was done for moves:

In [ ]:
abilities_cindex_distro = abilities_cindex[abilities_cindex['n_abilityset'] > 1]
abilities_cindex_distro.describe(percentiles = [n/20 for n in range(1, 20)])

Some brief notes:
- About 20% of abilities are not useful competitively, and thus have 0 cindex, and slightly more have 100 cindex

Now we need to remove zero values, name our new cindex_distro something sensible, and look at the describe and histogram methods to figure out explicit cutoffs for our aggregate features:

In [ ]:
cindex_distro_ability = abilities_cindex_distro.loc[abilities_cindex_distro['cindex'] > 0, 'cindex']
cindex_distro_ability.describe(percentiles = [n/20 for n in range(1, 20)])

In [ ]:
cindex_distro_ability.plot.hist(by='cindex')
plt.show()

This isn't relatively smooth exponential decay like the moves cindex was, so I won't use a log transform.  It's probably better to use pd.cut to look at the jumps like we did for the move power distro.

In [ ]:
cindex_distro_ability.plot.hist(by='cindex', bins=20)
plt.show()

In [ ]:
pd.cut(cindex_distro_ability, 20).value_counts().sort_index()

Ability cutoffs:
- 0 (basically, is it competitive at all?)
- 47.5 (pretty big cutoff, leading to a whole new "island")
- 65.0 (the final island before the big top)
- 73.75
- 95.625 (elite abilities)

<a id="ability_cutoff"></a>
### Aggregate Feature: Ability Cutoff

To start with, since pokemon have multiple abilities to choose from, we need to find out the best ability of each pokemon according to the ability cindexes.

In [ ]:
abilitylist = list(cindex_distro_ability.index)
cindex_encoder = pokemon_abilities_df.loc[:, abilitylist + list(abilities_cindex.loc[(abilities_cindex['cindex'] == 100) & (abilities_cindex['n_abilityset'] == 1), :].index)].copy()
for ability in abilitylist:
    cindex_encoder.loc[cindex_encoder[ability] == 1, ability] = abilities_cindex.loc[ability, 'cindex']
bestability =  cindex_encoder.max(axis=1)
bestability.shape

Now, we can simply create the feature according to the cutoffs we just decided on, where pokemon whose best ability is higher than the associated cutoff will be one-hot encoded as a 1, and those who don't have any ability as high as the indicated cutff will be encoded as a 0.

In [ ]:
shrinkfeatures_df['Ability Cutoff 1'] = bestability.where(bestability == 0, 1)
shrinkfeatures_df['Ability Cutoff 1'].value_counts()

In [ ]:
shrinkfeatures_df['Ability Cutoff 2'] = bestability.where(bestability < 47.5, 1).where(bestability >= 47.5, 0)
shrinkfeatures_df['Ability Cutoff 2'].value_counts()

In [ ]:
shrinkfeatures_df['Ability Cutoff 3'] = bestability.where(bestability < 65, 1).where(bestability >= 65, 0)
shrinkfeatures_df['Ability Cutoff 3'].value_counts()

In [ ]:
shrinkfeatures_df['Ability Cutoff 4'] = bestability.where(bestability < 73.75, 1).where(bestability >= 73.75, 0)
shrinkfeatures_df['Ability Cutoff 4'].value_counts()

In [ ]:
shrinkfeatures_df['Ability Cutoff 5'] = bestability.where(bestability < 82.5, 1).where(bestability >= 82.5, 0)
shrinkfeatures_df['Ability Cutoff 5'].value_counts()

In [ ]:
shrinkfeatures_df['Ability Cutoff 6'] = bestability.where(bestability < 95.625, 1).where(bestability >= 95.625, 0)
shrinkfeatures_df['Ability Cutoff 6'].value_counts()

<a id="other_ability"></a>
### Other Ability Features

Because it's not clear whether having a one-hot encoded feature for abilities, as we already have, is better than having a feature for abilities that just records the cindex of the best ability of each pokemon, we can also have a best ability cindex feature very easily:

In [ ]:
shrinkfeatures_df['Best Ability'] = bestability
shrinkfeatures_df['Best Ability'].describe(percentiles = [n/20 for n in range(1, 20)])

I also want to done another feature which doesn't include any abilities with 100 cindex, since this gives the model more information about a few things in combination with the other ability features:
- who has 100 cindex abilities that aren't unique (since they would have 100 in bestability but wouldn't have a "unique powerful ability")
- who has 100 cindex abilities that are unique
- how good abilities are that aren't 100 cindex, making a more interesting and diverse distro

It's possible that this more convoluted feature won't end up being used in the final model, but it's easy to make since we simply have to adapt our code from before by adding some restrictions, so why not.

In [ ]:
abilitylist_non100 = list(cindex_distro_ability[cindex_distro_ability < 100].index)
cindex_encoder_non100 = pokemon_abilities_df.loc[:, abilitylist_non100].copy()
for ability in abilitylist_non100:
    cindex_encoder_non100.loc[cindex_encoder_non100[ability] == 1, ability] = cindex_distro_ability[ability]
shrinkfeatures_df['Best Ability <100'] = cindex_encoder_non100.max(axis=1)
shrinkfeatures_df['Best Ability <100'].describe(percentiles = [n/20 for n in range(1, 20)])

And finally, we of course need our unique powerful ability feature. We shouldn't forget from our investigation in abilities_cindex that Dauntless Shield, Unseen Fist and RKS System are really unique powerful abilities even though they won't be listed as such since their n_abilityset is misleadingly listed as being higher than 1, so let's change abilities_cindex to fix this:

In [ ]:
abilities_cindex.loc[['Dauntless Shield', 'Unseen Fist', 'RKS System'], ['n_abilityset', 'n_competitive']] = 1
abilities_cindex.loc[['Dauntless Shield', 'Unseen Fist', 'RKS System'], :]

In [ ]:
shrinkfeatures_df['Unique Powerful Ability'] = pokemon_abilities_df.loc[:, list(abilities_cindex.loc[(abilities_cindex['cindex'] == 100) & (abilities_cindex['n_abilityset'] == 1), :].index)].sum(axis=1)
shrinkfeatures_df['Unique Powerful Ability'].value_counts()

There is still technically one more feature, but it's not really a new feature, it's just our target feature, format, as a number from 0 to 6 instead of a category, which may be imporant for some algorithms, especially since the tiers are ordinal, not just discrete.

In [ ]:
pokemon_df['format codes'] = pd.Categorical(pokemon_df['formats'], categories=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber']).codes
pokemon_df.head(10)

The final step is to put all of our data together in a single dataframe, combining pokemon_df (which contains the basic stats), pokemon_types_summary (which contains pokemon type information), and shrinkfeatures_df (which contains all manual and aggregate features we designed to summarize ability and move information):

In [ ]:
pokemon_data = pokemon_df.merge(pokemon_types_summary, on='name')
pokemon_data = pokemon_data.merge(shrinkfeatures_df, on='name')
pokemon_data.head()

In [ ]:
pokemon_data.columns

## Feature Selection and Visualizations

In [ ]:
pokemon_data.shape

In [ ]:
pokemon_data.select_dtypes(include='number').shape

<a id="removing_unnecessary"></a>
### Removing Obviously Unnecessary Features

We've narrowed down our features tremendously, going from hundreds of moves and abilities to having a total numeric feature count of 66. However, that's still not great for a multi-class classification problem given that it's only a bit less than one tenth of our sample size (738), and we also have lots of redundant features (e.g. many values for cutoff and coverage moves), so we can narrow our space of important features even more.

For starters, weight and height are rarely if ever used in pokemon, and whatever use they have would be so small that it would be completely overshadowed by other competitive attributes like the 6 major stats, the type, and features related to moves and abilities, so we can certainly get rid of weight and height.

We also designed our resistance index feature to summarize type information related to weaknesses, resistances and immunities, so having separate features for weaknesses, resistances and immunities taken out of context from their overall summary and relationship is not likely to be helpful (so we will remove all type summary information except the resistance index). The other one to remove is "STAB", since this feature (simply indicating whether a pokemon has 1 or 2 types) is not even close to impactful enough to predict the competitive viability of a pokemon (feel free to visit my EDA experiments if you doubt this since I tested the correlations of many features which isn't useful enough to put in this notebook but it made clear that many features are useless and most of them are much less impactful than the best features).

Of course we wouldn't include format codes as a feature since that is a version of the target feature.

The next major decision we have is which of the cutoff and coverage features to remove for moves and abilities. Probably the best we can do this is looking at their correlation with the target feature (in this case treated as a number: the ordinal format codes we just created) to see which one is likely to correlate best with competitive viability. We only need to use one cutoff and coverage feature from each of physical moves, special moves, and abilities, and we can also look at the correlations of other ability features we created which we are less certain about the usefulness of.

In [ ]:
pokemon_data.loc[:, ['Physical Cutoff 1', 'Physical Cutoff 2', 'Physical Cutoff 3', 'Physical Cutoff 4', 'Physical Cutoff 5', 'Physical Cutoff 6']].corrwith(pokemon_data['format codes'])

In [ ]:
pokemon_data.loc[:, ['Physical Coverage 1', 'Physical Coverage 2', 'Physical Coverage 3', 'Physical Coverage 4']].corrwith(pokemon_data['format codes'])

In [ ]:
pokemon_data.loc[:, ['Special Cutoff 1', 'Special Cutoff 2', 'Special Cutoff 3', 'Special Cutoff 4', 'Special Cutoff 5', 'Special Cutoff 6', 'Special Cutoff 7']].corrwith(pokemon_data['format codes'])

In [ ]:
pokemon_data.loc[:, ['Special Coverage 1', 'Special Coverage 2', 'Special Coverage 3', 'Special Coverage 4', 'Special Coverage 5', 'Special Coverage 6', 'Special Coverage 7', 'Special Coverage 8', 'Special Coverage 9', 'Special Coverage 10']].corrwith(pokemon_data['format codes'])

In [ ]:
pokemon_data.loc[:, ['Ability Cutoff 1', 'Ability Cutoff 2', 'Ability Cutoff 3', 'Ability Cutoff 4', 'Ability Cutoff 5', 'Ability Cutoff 6', 'Best Ability', 'Best Ability <100', 'Unique Powerful Ability']].corrwith(pokemon_data['format codes'])

First of all, it's just generally interesting that abilities (at least discretely) have less correlation with competitive viability than movesets. One reason could be that since abilities are far fewer, they may contain a lot more uninformative "0's" whereas movesets almost never have 0's in lower cutoffs.

For most of these, the choice is pretty obvious: the **number 3** version of each feature is the natural elbow point, so it's the one we'll use. This even includes the Ability Cutoff feature, but our Best Ability <100 feature has a NEGATIVE correlation with competitive viability. It was already suspected to be a pretty convoluted feature, so we should just get rid of it.

On another note, the special moves get strangely correlated to competitive viability the higher they get in power (as in the Special Coverage feature), which is strange because I know that's not really how moves are used in competitive play (maybe it just has to do with a handful of legendaries having powerful moves in that direction). So we should look at which moves are being ranked so highly, and calculate the median cindex for each cutoff group:

**Group 10:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=146) & (moves_cindex['category']=='Special'), :]

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=146) & (moves_cindex['category']=='Special'), 'cindex'].median(axis=0)

This is indeed very informative, because although more powerful pokemon may happen to have moves this powerful more frequently, the reality is that almost all of them have very low cindex, so choosing a cutoff this high is very uninformative for competitive viability, especially with unseen data.

**Group 9:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=139) & (moves_cindex['power']<146) & (moves_cindex['category']=='Special'), :]

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=139) & (moves_cindex['power']<146) & (moves_cindex['category']=='Special'), 'cindex'].median(axis=0)

Boomburst is a great move, but it's not a coverage move because it's normal type, so that's not what it makes it good, and thus it isn't very informative for this feature (it can be covered in the move cutoff feature). The other moves here are generally weak.

**Group 8:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=118) & (moves_cindex['power']<139) & (moves_cindex['category']=='Special'), :]

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=118) & (moves_cindex['power']<139) & (moves_cindex['category']=='Special'), 'cindex'].median(axis=0)

This is starting to look like possibly a good feature, but there are a lot of unique powerful moves here which are less informative due to sampple size of n_learnset=1.

**Group 7:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=104) & (moves_cindex['power']<118) & (moves_cindex['category']=='Special'), :]

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=104) & (moves_cindex['power']<118) & (moves_cindex['category']=='Special'), 'cindex'].median(axis=0)

This one is a whole different level of good, as most of these moves are informative. Group 7 is the best so far.

**Group 6:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=97) & (moves_cindex['power']<104) & (moves_cindex['category']=='Special'), :]

This is a huge concentration of low sample size moves, so I think it's not potentially informative. Group 7 is still the best.

**Group 5:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=76) & (moves_cindex['power']<97) & (moves_cindex['category']=='Special'), :]

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=76) & (moves_cindex['power']<97) & (moves_cindex['category']=='Special'), 'cindex'].median(axis=0)

Unfortunately, I'm inclined to think this one **TRULY IS** informative, knowing a lot of competitive moves are in this category, but its median cindex isn't huge (though it's not small either), there might just be a lot of garbage here too.

**Group 4:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=55) & (moves_cindex['power']<76) & (moves_cindex['category']=='Special'), :]

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=55) & (moves_cindex['power']<76) & (moves_cindex['category']=='Special'), 'cindex'].median(axis=0)

This one is garbage and I suspect all below it will be as well

**Group 3:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=48) & (moves_cindex['power']<55) & (moves_cindex['category']=='Special'), :]

This one is obviously no good.

**Group 2:**

In [ ]:
moves_cindex.loc[(moves_cindex['power']>=34) & (moves_cindex['power']<48) & (moves_cindex['category']=='Special'), :]

So the rest of the lower powered moves are not competitively viable, thus my two choices for special coverage seem pretty obvious (5 or 7). I will choose just one of them (the lower one, group 5) for the sake of conciseness and because it includes both the informative groups whereas choosing 7 will exclude the informative group 5 which contains many of the most common coverage moves. Including anything below group 5 makes no sense.

<a id="pca"></a>
### Principal Component Analysis

Before performing our likely far more successful and useful PCA, on account of having such a smaller number of features, let's remind ourselves of all the columns that we will be removing from consideration in our current iteration of pokemon_data:

'weight', 'height', 'format codes', 'Weaknesses', 'Strong Weaknesses', 'Resists', 'Strong Resists', 'Immune', 'STAB', 'Physical Cutoff 1', 'Physical Cutoff 2', 'Physical Cutoff 4', 'Physical Cutoff 5', 'Physical Cutoff 6', 'Physical Coverage 1', 'Physical Coverage 2', 'Physical Coverage 4', 'Special Cutoff 1', 'Special Cutoff 2', 'Special Cutoff 4', 'Special Cutoff 5', 'Special Cutoff 6', 'Special Cutoff 7', 'Special Coverage 1', 'Special Coverage 2', 'Special Coverage 3', 'Special Coverage 4', 'Special Coverage 6', 'Special Coverage 7', 'Special Coverage 8', 'Special Coverage 9', 'Special Coverage 10', 'Ability Cutoff 1', 'Ability Cutoff 2', 'Ability Cutoff 4', 'Ability Cutoff 5', 'Ability Cutoff 6', 'Best Ability <100'

In [ ]:
pokemon_data_pca = pokemon_data.drop(columns=['weight', 'height', 'format codes', 'Weaknesses', 'Strong Weaknesses', 'Resists',
                                              'Strong Resists', 'Immune', 'STAB', 'Physical Cutoff 1', 'Physical Cutoff 2',
                                              'Physical Cutoff 4', 'Physical Cutoff 5', 'Physical Cutoff 6',
                                              'Physical Coverage 1', 'Physical Coverage 2', 'Physical Coverage 4',
                                              'Special Cutoff 1', 'Special Cutoff 2', 'Special Cutoff 4',
                                              'Special Cutoff 5', 'Special Cutoff 6', 'Special Cutoff 7',
                                              'Special Coverage 1', 'Special Coverage 2', 'Special Coverage 3',
                                              'Special Coverage 4', 'Special Coverage 6', 'Special Coverage 7',
                                              'Special Coverage 8', 'Special Coverage 9', 'Special Coverage 10',
                                              'Ability Cutoff 1', 'Ability Cutoff 2', 'Ability Cutoff 4', 'Ability Cutoff 5',
                                              'Ability Cutoff 6', 'Best Ability <100'])

pokemon_data_nonnumeric = pokemon_data_pca.select_dtypes(include='object', exclude='number')
pokemon_data_numeric = pokemon_data_pca.select_dtypes(include='number')
data_scaled = scale(pokemon_data_numeric)
pokemon_data_scaled = pd.DataFrame(data_scaled, index=pokemon_data.index, columns=pokemon_data_numeric.columns)
pokemon_data_fit = PCA().fit(pokemon_data_scaled)

plt.subplots(figsize=(10, 6))
plt.plot(pokemon_data_fit.explained_variance_ratio_.cumsum())
plt.plot(pokemon_data_fit.explained_variance_ratio_)
plt.xlabel('Component #')
plt.ylabel('Cumulative ratio variance')
plt.title('Cumulative variance ratio explained by PCA components for aggregated pokemon stats');

In [ ]:
pokemon_data_fit.components_.shape

Only 28 components is amazing compared to the 100's of components we had previously, and is FAR more useful.

In [ ]:
pokemon_data_compdf = pd.DataFrame(pokemon_data_fit.components_, columns=pokemon_data_numeric.columns)
pokemon_data_compdf.head()

We can also zoom in on the scree plot a little more to get a more precise idea of where the elbow is.

In [ ]:
plt.subplots(figsize=(10, 6))
plt.plot(pokemon_data_fit.explained_variance_ratio_)
plt.xlabel('Component #')
plt.ylabel('Cumulative ratio variance')
plt.title('Scree plot for pokemon stats PCA');

The elbow is around 6 or 7 components, so that's probably all we have to look at deeply for our analysis, as after that the components become individually quite insignificant.

**First component:**

In [ ]:
plt.plot(pokemon_data_compdf.loc[0, :].sort_values(ascending=False).head(14))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.plot(pokemon_data_compdf.loc[0, :].sort_values().head(14))
plt.xticks(rotation = 90)
plt.show()

So this feature mainly captures the stats and move access of pokemon, what overall threat they represent, which makes sense as the strongest component. The few negative features are cheesy tactics that don't depend on stats so much, such as priority moves and entry hazards, so it makes sense why they'd be negatively correlated here.

**Second component:**

In [ ]:
plt.plot(pokemon_data_compdf.loc[1, :].sort_values(ascending=False).head(14))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.plot(pokemon_data_compdf.loc[1, :].sort_values().head(14))
plt.xticks(rotation = 90)
plt.show()

This feature mainly captures the physical power of a pokemon (physical movesets, attack and defense), combined with entry hazard setting ability on the positive side, and on the negative side are mainly status oriented moves and special attack and abilities (which may indeed be opposite to a more direct physically oriented pokemon)

**Third component:**

In [ ]:
plt.plot(pokemon_data_compdf.loc[2, :].sort_values(ascending=False).head(14))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.plot(pokemon_data_compdf.loc[2, :].sort_values().head(14))
plt.xticks(rotation = 90)
plt.show()

This is a very interesting feature which heavily prioritizes having a unique powerful ability, rather than a good overall best ability or high ability cutoff point (including weather set or gimmick). It also looks favorably upon resistance index, pivot, speed and misc status, so it might be a feature that's measuring a certain kind of pivot that has a unique ability

**Fourth component:**

In [ ]:
plt.plot(pokemon_data_compdf.loc[3, :].sort_values(ascending=False).head(14))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.plot(pokemon_data_compdf.loc[3, :].sort_values().head(14))
plt.xticks(rotation = 90)
plt.show()

This is a defensive feature, prioritizing entry hazard setting, def, spd, cleric and hp recovery, and hp itself. It is opposed to fast pivots with priority moves and hazard removal, as well as setup sweepers, who are perhaps the opposite.

**Fifth component:**

In [ ]:
plt.plot(pokemon_data_compdf.loc[4, :].sort_values(ascending=False).head(14))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.plot(pokemon_data_compdf.loc[4, :].sort_values().head(14))
plt.xticks(rotation = 90)
plt.show()

Here is a feature that is a bit stranger and harder to interpret, it really prioritzes HP Drain and Item removal, which is an odd combination, and to a lesser extent, physical moveset, priority, setup, hp recovery and cleric and removal deterrent are prioritized. It's heavily against unique powerful moves, and to a lesser extent, pivoting, weather setting and general stats (so it's more about the useful moves and abilities a pokemon has to stay alive perhaps, regardless of stats, or perhaps about physical sweepers that use priority and recovery moves like drain punch, which may be a common trope).

**Sixth component:**

In [ ]:
plt.plot(pokemon_data_compdf.loc[5, :].sort_values(ascending=False).head(14))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.plot(pokemon_data_compdf.loc[5, :].sort_values().head(14))
plt.xticks(rotation = 90)
plt.show()

This is another feature measuring intangibles, like resistance index and removal deterrent especially, as well as misc status moves and ability rating, positively, and negatively correlated to weather gimmick and setting (perhaps because those are special abilities) and even more often hp, and any sort of hp recovery, drain, cleric, etc. (so perhaps it's intangles that keep pokemon alive other than direct hp and hp recovery, such as their type, ability and status moves).

**Seventh component:**

In [ ]:
plt.plot(pokemon_data_compdf.loc[6, :].sort_values(ascending=False).head(14))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.plot(pokemon_data_compdf.loc[6, :].sort_values().head(14))
plt.xticks(rotation = 90)
plt.show()

This feature seems to be about staying alive and setting up, due to high defense, special defense, entry hazards and removal, setup and hp recovery, and it is negatively correlated with other moves the pokemon has (physical and special).

Now, I'd look to do pairplots of these seven components, so there are not too many, to get an idea of the actual shape of the data

In [ ]:
pokemon_data_comp = pokemon_data_fit.transform(pokemon_data_scaled)
pokemon_data_pcadf = pd.DataFrame(pokemon_data_comp, index=pokemon_data.index)
pokemon_data_pcadf['formats'] = pokemon_data['formats']
pokemon_data_pcadf.head()

In [ ]:
pairplot_pcadf_list = [n for n in range(7)]
pairplot_pcadf_list.append('formats')
sns.pairplot(pokemon_data_pcadf.loc[:, pairplot_pcadf_list], hue='formats', hue_order=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber'])
plt.show()

On the outer left and upward edges of the charts, we can see correlations with the first components which do cluster pretty well around the tier due to the neater cascade of colors. This makes sense since that first component was based a lot on overall stats, which is expected to correlate well with overall competitive viability. However, most of the other charts moving down and towards the right are just a mess of color, so don't correlate well to tier at all. That's not necessarily bad though, as they may help more with making differentiations within tier, rather than predicting it as directly as stats.

The general lack of clustering is also interesting, as it indicates that using PCA for our clustering algorithms later probably won't be helpful compared to just clustering the data directly. Also, it's likely that more artificial, forced clustering techniques like K-Means will work better than techniques that find more natural clusters like density-based clustering, since there aren't particularly good natural clusters (and it's more likely to just find outliers, as we will discover later).

<a id="data_prep"></a>
### Preparing the Data for OOTB Modeling

Now we're going to try some out of the box modeling, not significantly optimized for performance but more to get feedback about the features in our data. In particular, we want to look at the general performance of some different algorithms, as well as what coefficients and importances are assigned to the features so that we can see what features are putting in the most work and which are not very useful.

To do that, we need to prepare our data in the form of an X and y variable typically used in machine learning.

For our y variable, we will need to use several versions to have an idea of what target variable is best to use.  We could try up to 6:
- competitive or not competitive (binary classification)
- 4 classes: (ZU, PU, NU+RU+UU, OU+Uber)
- all 7 classes at once
And each of those have one variation with the tierchange work we did, and one variation without it

And I can also do a linreg variation on each one of those (classic lasso), with the exception of the 2 classes version, since there is no point of doing linear regression in such a case.

Let's start with getting back our old formats target variable from before we modified it with tierchange_df:

In [ ]:
oldformats = pd.read_csv('./data/pokemon_df.csv', index_col="name")['formats']
oldformats[oldformats == 'LC'] = 'ZU'
oldformats[oldformats == 'NFE'] = 'ZU'
oldformats[oldformats == 'Untiered'] = 'ZU'
oldformats[oldformats == 'PUBL'] = 'NU'
oldformats[oldformats == 'NUBL'] = 'RU'
oldformats[oldformats == 'RUBL'] = 'UU'
oldformats[oldformats == 'UUBL'] = 'OU'
oldformats.unique()

Then we need to make a column for our oldformats in both string and linear regression formats, and we also need to make a dataframe to hold all of our different target variables to select from.

In [ ]:
pokemon_data['oldformats'] = oldformats
pokemon_data['oldformat codes'] = pd.Categorical(pokemon_data['oldformats'], categories=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber']).codes

X = pokemon_data.drop(columns=['weight', 'height', 'Weaknesses', 'Strong Weaknesses', 'Resists',
                                'Strong Resists', 'Immune', 'STAB', 'Physical Cutoff 1', 'Physical Cutoff 2',
                                'Physical Cutoff 4', 'Physical Cutoff 5', 'Physical Cutoff 6',
                                'Physical Coverage 1', 'Physical Coverage 2', 'Physical Coverage 4',
                                'Special Cutoff 1', 'Special Cutoff 2', 'Special Cutoff 4',
                                'Special Cutoff 5', 'Special Cutoff 6', 'Special Cutoff 7',
                                'Special Coverage 1', 'Special Coverage 2', 'Special Coverage 3',
                                'Special Coverage 4', 'Special Coverage 6', 'Special Coverage 7',
                                'Special Coverage 8', 'Special Coverage 9', 'Special Coverage 10',
                                'Ability Cutoff 1', 'Ability Cutoff 2', 'Ability Cutoff 4', 'Ability Cutoff 5',
                                'Ability Cutoff 6', 'Best Ability <100', 'formats', 'generation',
                                'format codes', 'oldformats', 'oldformat codes'])

y_df = pd.DataFrame(pokemon_data[['formats', 'format codes', 'oldformats', 'oldformat codes']], index=pokemon_data.index, columns=['formats', 'format codes', 'oldformats', 'oldformat codes'])
y_df['formats4'] = y_df['formats'].replace({'ZU':'Not c', 'PU': 'Low c', 'NU': 'Mid c', 'RU': 'Mid c', 'UU': 'Mid c', 'OU': 'High c', 'Uber': 'High c'})
y_df['format codes4'] = y_df['format codes'].replace({3:2, 4: 2, 5:3, 6:3})
y_df['oldformats4'] = y_df['oldformats'].replace({'ZU':'Not c', 'PU': 'Low c', 'NU': 'Mid c', 'RU': 'Mid c', 'UU': 'Mid c', 'OU': 'High c', 'Uber': 'High c'})
y_df['oldformat codes4'] = y_df['oldformat codes'].replace({3:2, 4: 2, 5:3, 6:3})
y_df['formats2'] = y_df['formats'].replace({'ZU':'No', 'PU': 'Yes', 'NU': 'Yes', 'RU': 'Yes', 'UU': 'Yes', 'OU': 'Yes', 'Uber': 'Yes'})
y_df['oldformats2'] = y_df['oldformats'].replace({'ZU':'No', 'PU': 'Yes', 'NU': 'Yes', 'RU': 'Yes', 'UU': 'Yes', 'OU': 'Yes', 'Uber': 'Yes'})
y_df

<a id="logistic_l1"></a>
### Logistic Regression with L1 norm

The first model we're going to use is logistic regression with an l1 norm, i.e. Lasso, which is especially inclined to penalize features which aren't contributing to the model by making their coefficients 0, which is useful for the feature selection we are engaged in.

First we'll try it with all 7 classes:

In [ ]:
pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipe.fit(X, y_df['formats'])

y_pred = pipe.predict(X)

precision_recall_fscore_support(y_df['formats'], y_pred, labels=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber'])

This is interesting, so it's doing very well on ZU and Ubers (the extremes, which make sense as they are the easiest), and especially poorly on NU and on the recall for NU, RU and UU (which makes sense, they have the fewest examples plus they are more middling so they require more nuance to understand than the extremes).

Not only does this support the idea of a two-stage model, where the model might have a chance for more differentiations, it also STRONGLY supports a 4 class approach, since all three of our weakest classes would be included in a single larger class that I think would be far easier for the model to recognize than any of the individual classes

Let's look at the coefficients though, which is the main reason we created this model:

In [ ]:
model1_coef = pd.DataFrame(pipe._final_estimator.coef_, index=pipe._final_estimator.classes_, columns=X.columns)
feature_zeroes = (model1_coef==0).sum(axis=0)
feature_zeroes.sort_values(ascending=False)

Most of all, L1 Logistic Regression found that **Physical Cutoff**, **Best Ability**, **HP Recovery**, **Ability Cutoff** and **Cleric** were not important for competitive viability, but it needs to be kept in mind that none of these were always considered unimportant (they just had the most zero values totalled over the classes), and the model did not perform particularly well (so it may just not be seeing their importance sufficiently).

We also need to try this L1 Logistic Regression with the 4 class and binary classification methods.

In [ ]:
pipe4 = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipe4.fit(X, y_df['formats4'])

y_pred4 = pipe4.predict(X)
precision_recall_fscore_support(y_df['formats4'], y_pred4, labels=['Not c', 'Low c', 'Mid c', 'High c'])

These are notably better scores, with no class below 50% in the f1 score.

In [ ]:
model_formats4_coef = pd.DataFrame(pipe4._final_estimator.coef_, index=pipe4._final_estimator.classes_, columns=X.columns)
feature_zeroes = (model_formats4_coef==0).sum(axis=0)
feature_zeroes.sort_values(ascending=False)

**Best Ability** and **Physical Cutoff** are still slightly struggling given that they got a zero for some classes, but overall there aren't many zeros being given for any of the classes so none of the features seem to doing that bad, though the two I mentioned continue to be under the most suspicion.

In [ ]:
pipe2 = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipe2.fit(X, y_df['formats2'])

y_pred2 = pipe2.predict(X)
precision_recall_fscore_support(y_df['formats2'], y_pred2, labels=['No', 'Yes'])

This is performing very well indeed, and I think **gives the strongest support by far of splitting this task up into a two-stage model**, where we do binary classification first (this will likely be very high accuracy, given that we got around 90% in both classes with an out of the box model with no tuning), and then trying to figure out which of the competitive classes the remaining pokemon belong in (either split into the original 3, or perhaps low/mid/high as previously proposed, or something else suggested to us by clustering)

We also want to remember, when actually modeling, that **we may want to use weighted precision and recall matrices, given unbalanced class sizes** (this is especially important with more than two classes, I think), for now it's not that big of a deal since we aren't focusing on score very much, just trying to gain insight (and perhaps f score offers something like that).

In [ ]:
model_formats2_coef = pd.DataFrame(pipe2._final_estimator.coef_, columns=X.columns)
model_formats2_coef.T.sort_values(by=0, ascending=False)

We are consistently hearing that **Physical Cutoff** and **Best Ability** are not relevant to assessing the competitive viability of pokemon, at least from this model, which is interesting. **Unique Powerful Ability** also gave no information to this model.

Unsurprisingly the stats and type information were important. The others seem less consistent but Ability Cutoff, Item Removal, Cleric, and HP Drain/Recovery, Physical Coverage, Priority and Entry Hazard setting held some importance in this model, and Hazard Removal, Unique Powerful Moves, Special Coverage, Pivot moves and Weather Setting abilities predicted against competitive viability (most of which is pretty strange).

That said, **we may want to pay attention to the possible uselessness of the cutoff and ability features**, **especially if** they have other problems, like **strong correlation with other features** (such as the coverage features which did turn out to be more informative)

Let's try again with the old formats that haven't been tampered with:

In [ ]:
pipeold = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipeold.fit(X, y_df['oldformats'])

y_predold = pipeold.predict(X)
precision_recall_fscore_support(y_df['oldformats'], y_predold, labels=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber'])

The extremes still do very well here, with Uber doing significantly better than last time we tried, and none of our classes doing nearly as bad as our worst classes last time, but not much better performance on the whole especially with PU doing far worse.

In [ ]:
model_oldformats_coef = pd.DataFrame(pipeold._final_estimator.coef_, index=pipeold._final_estimator.classes_, columns=X.columns)
feature_zeroes = (model_oldformats_coef==0).sum(axis=0)
feature_zeroes.sort_values(ascending=False)

**Physical Cutoff** and **Best Ability** again having problems, along with Special Coverage, Ability Cutoff, Setup, Entry Hazards.  We'll have to see how consistent these are as trends to derive much insights from it.

In [ ]:
pipeold4 = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipeold4.fit(X, y_df['oldformats4'])

y_predold4 = pipeold4.predict(X)
precision_recall_fscore_support(y_df['oldformats4'], y_predold4, labels=['Not c', 'Low c', 'Mid c', 'High c'])

This is actually doing far worse than our tierchange_df formats for four classes, and it's because for tierchange_df, PU was very large, but in oldformats PU is much too small so the model performs terribly on it. A workaround is dividing up 4 classes differently, combing PU and NU with low competitive (low c) and having mid competitive (mid c) be only RU and UU, to balance class sizes (but we'll only make that adjustment for oldformats).

In [ ]:
y_df['oldformats4'] = y_df['oldformats'].replace({'ZU':'Not c', 'PU': 'Low c', 'NU': 'Low c', 'RU': 'Mid c', 'UU': 'Mid c', 'OU': 'High c', 'Uber': 'High c'})
y_df['oldformat codes4'] = y_df['oldformat codes'].replace({2:1, 3:2, 4: 2, 5:3, 6:3})

In [ ]:
pipeold4 = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipeold4.fit(X, y_df['oldformats4'])

y_predold4 = pipeold4.predict(X)
precision_recall_fscore_support(y_df['oldformats4'], y_predold4, labels=['Not c', 'Low c', 'Mid c', 'High c'])

This split barely created any improvment in low competitive. We can also try splitting low competitive into PU, NU AND RU, and mid competitive will be UU and OU and the Ubers can stand alone in high competitive due to how extreme and recognizable they often are.

In [ ]:
y_df['oldformats4'] = y_df['oldformats'].replace({'ZU':'Not c', 'PU': 'Low c', 'NU': 'Low c', 'RU': 'Low c', 'UU': 'Mid c', 'OU': 'Mid c', 'Uber': 'High c'})
y_df['oldformat codes4'] = y_df['oldformat codes'].replace({2:1, 3:1, 4: 2, 5:2, 6:3})

In [ ]:
pipeold4 = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipeold4.fit(X, y_df['oldformats4'])

y_predold4 = pipeold4.predict(X)
precision_recall_fscore_support(y_df['oldformats4'], y_predold4, labels=['Not c', 'Low c', 'Mid c', 'High c'])

This actually lead to remarkable improvements, with low competitive still not being quite as good as it was for tierchange_df 4 classes, but Ubers/high competitive actually being even better (a sign that we might have good reason to isolate Ubers).

In [ ]:
model_oldformats4_coef = pd.DataFrame(pipeold4._final_estimator.coef_, index=pipeold4._final_estimator.classes_, columns=X.columns)
feature_zeroes = (model_oldformats4_coef==0).sum(axis=0)
feature_zeroes.sort_values(ascending=False)

There aren't many 0's here at all, but Best Ability and Physical Cutoff still have one zero each. Priority has the most but has never been consistently bad so I'm highly doubtful of that meaning anything significant.

Finally, oldformats binary classification:

In [ ]:
pipeold2 = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l1', solver = 'liblinear', random_state=42))

pipeold2.fit(X, y_df['oldformats2'])

y_predold2 = pipeold2.predict(X)
precision_recall_fscore_support(y_df['oldformats2'], y_predold2, labels=['No', 'Yes'])

This is not terrible, but it is notably worse in accuracy than using tierchange_df formats, since tierchange_df must have made ZU more distinct by removing even slightly competitive pokemon into PU.

In [ ]:
model_oldformats2_coef = pd.DataFrame(pipeold2._final_estimator.coef_, columns=X.columns)
model_oldformats2_coef.T.sort_values(by=0, ascending=False)

Nothing was 0 this time, so I'm not sure we have a good enough justification to remove anything yet, but Ability features in general and Physical Cutoff are under suspicion.

Special Coverage and Hazard Removal have also been correlated surprisingly against competitiveness in both the binary models, which is also noteworthy to keep an eye on. Unique Powerful Move has also been consistently slightly anti-competitive. Most of the others have been pretty inconsistent on the negative side, either meaning very little or sometimes meaning a lot and sometimes meaning close to nothing or even fluctuation to slightly positive like with Weather Set.

The positively correlated features have been quite consistent though: stats, type information, HP Drain/Recovery, Item Removal and Entry Hazards, and sometimes Cleric and Ability Cutoff.

In the EDA experiments 2 notebook, we tried doing Linear Regression with L1 Lasso penalty and it found almost all coefficients to be zero and had an r-squared score that was on par with random guessing. Given time constraints, and our previous lack of success with looking at pearson correlations to the linear target feature codes, I think we are just going to leave linear regression out of this project, as it has proven itself to be completely useless, whereas logistic regression has provided noteworthy insight and not terrible accuracy. That means we're done with regression, and we can move on to decision trees (in particular, random forest, which is my personal favorite model for this project, but we'll see what performs the best). The reason I like random forest is that it's nonlinear, deals with combinations of features really well, and it simulates the random nature of pokemon (battles have a huge element of randomness, you don't know what you'll come up against so to be good you have to be able to handle randomness and be prepared for it).

<a id="random_forest"></a>
### Random Forest Feature Importances

I think we WILL use grid search cv for this since we have no idea what a good number of trees is, and it's not hard to try 6 values for that and just pick what turned out best, but we won't worry about in-depth hyperparameter optimization. We will save that completely for pre-processing, and now we instead want to look at feature performances and the general performance of the algorithm compared to unoptimized logistic regression. There is no need to scale features for random forest so we won't even bother, thus we don't really even need a pipeline, we can just plug RF right into Grid Search. We'll probably just set cross validation to 2 folds to minimize its impact (probably we will not want too many cv folds in our final model anyways due to having some class sizes that are quite small).

We also need to use a train-test split because random forest heavily overfits on training data without advanced optimization on our part that we don't have time for in this OOTB modeling, so it should be kept in mind that it's performance is being heavily disadvantaged compared to logistic regression since we did not split for that algorithm.

In [ ]:
#setting up our different tree counts for all models
n_est = [int(n) for n in np.logspace(start=1, stop=3, num=6)]
grid_params = {'n_estimators': n_est}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_df['formats'],
                        test_size=0.25, random_state=42,
                        stratify=y_df['formats'].values)

random_forest_classifier = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(random_forest_classifier, param_grid=grid_params, cv=2, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_, rf_grid.best_score_

In [ ]:
y_predrf_train = rf_grid.predict(X_train)
y_predrf_test = rf_grid.predict(X_test)
precision_recall_fscore_support(y_train, y_predrf_train, labels=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber']), precision_recall_fscore_support(y_test, y_predrf_test, labels=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber'])

This is quite realistic; it's doing really well on the training data (overfitting a lot, which we will want to pay attention to when modeling), but on the test data not quite so well, so the results are more similar to logistic regression (though since I didn't use testing data for logistic regression, logistic regression might well have done even worse on it). In any case, let's look at feature importances this time:

In [ ]:
plt.subplots(figsize=(10, 5))
imps = rf_grid.best_estimator_.feature_importances_
rf_feat_imps = pd.Series(imps, index=X.columns).sort_values(ascending=False)
rf_feat_imps.plot(kind='bar')
plt.xlabel('features')
plt.ylabel('importance')
plt.title('Best random forest classifier feature importances');

We see a nice tiering in the importance of features here:
- the most useful are the stats
- the next set of quite useful features are resistance index, cutoff and coverage move features, and best ability
- then the remaining features are much less useful, with the least useful being Weather Set, Unique Powerful Ability, Removal Deterrent, Ability Cutoff 3, Hazard Removal and Unique Powerful Move, while Weather Gimmick, Misc Status and Setup are still quite useful.

This makes me wonder about an idea of folding in some of these weaker features into some of our better features, like folding weather set into Best Ability or Weather Gimmick to simply enhance them while removing a relatively confusing and pointless extraneous feature. Unique Powerful Ability and Ability Cutoff 3 could arguably just be removed as Best Ability probably just captures the ability information much better. For some of the others, we could think carefully about the best options (e.g. it might be useful to meld HP Recovery and HP Drain)

Now we will try with 4 classes:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_df['formats4'],
                        test_size=0.25, random_state=42,
                        stratify=y_df['formats4'].values)

random_forest_classifier = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(random_forest_classifier, param_grid=grid_params, cv=2, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_, rf_grid.best_score_

In [ ]:
y_predrf4_train = rf_grid.predict(X_train)
y_predrf4_test = rf_grid.predict(X_test)
precision_recall_fscore_support(y_train, y_predrf4_train, labels=['Not c', 'Low c', 'Mid c', 'High c']), precision_recall_fscore_support(y_test, y_predrf4_test, labels=['Not c', 'Low c', 'Mid c', 'High c'])

Pretty similar performance to logistic regression, but maybe slightly worse in the 2nd highest class, though we had to train test split the data this time which may be the reason.

In [ ]:
plt.subplots(figsize=(10, 5))
imps = rf_grid.best_estimator_.feature_importances_
rf_feat_imps = pd.Series(imps, index=X.columns).sort_values(ascending=False)
rf_feat_imps.plot(kind='bar')
plt.xlabel('features')
plt.ylabel('importance')
plt.title('Best random forest classifier feature importances');

This remains nearly identical in overall narrative, and even many of the details.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_df['formats2'],
                        test_size=0.25, random_state=42,
                        stratify=y_df['formats2'].values)

random_forest_classifier = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(random_forest_classifier, param_grid=grid_params, cv=2, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_, rf_grid.best_score_

In [ ]:
y_predrf2_train = rf_grid.predict(X_train)
y_predrf2_test = rf_grid.predict(X_test)
precision_recall_fscore_support(y_train, y_predrf2_train, labels=['No', 'Yes']), precision_recall_fscore_support(y_test, y_predrf2_test, labels=['No', 'Yes'])

It did pretty well, not quite as well as logistic regression (but again we didn't split into training and test sets), but still very well, so it remains promosing.

In [ ]:
plt.subplots(figsize=(10, 5))
imps = rf_grid.best_estimator_.feature_importances_
rf_feat_imps = pd.Series(imps, index=X.columns).sort_values(ascending=False)
rf_feat_imps.plot(kind='bar')
plt.xlabel('features')
plt.ylabel('importance')
plt.title('Best random forest classifier feature importances');

It's still very similar to previous forests but even more weighted towards stats (which makes sense, you need certain stats to be in the higher tiers otherwise you will just get mowed down or be too passive to do anything).

This is a sign that we could expect any binary model we make to be heavily weighted towards stats, but then our secondary model which categorizes more finely will use the other features more closely with the stats.

Now let's get into oldformats:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_df['oldformats'],
                        test_size=0.25, random_state=42,
                        stratify=y_df['oldformats'].values)
random_forest_classifier = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(random_forest_classifier, param_grid=grid_params, cv=2, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_, rf_grid.best_score_

In [ ]:
y_predrfold_train = rf_grid.predict(X_train)
y_predrfold_test = rf_grid.predict(X_test)
precision_recall_fscore_support(y_train, y_predrfold_train, labels=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber']), precision_recall_fscore_support(y_test, y_predrfold_test, labels=['ZU', 'PU', 'NU', 'RU', 'UU', 'OU', 'Uber'])

In spite of the "better score" oldformats 7 class received than tierchange_df 7 classes did, the oldformats model actually performed notably worse on the precision recall matrix, having many outright 0 f scores. It may only be better on the training set, and thus I may want to question whether Grid Search CV might not always give the best model, it might be giving me overfitted garbage, and I may have to assess things based on a more manual search to try prevent overfitting, or perhaps other metrics and hyperparameters will help with that. But overall, I'd have to say oldformats performs terribly with 7 classes.

In [ ]:
plt.subplots(figsize=(10, 5))
imps = rf_grid.best_estimator_.feature_importances_
rf_feat_imps = pd.Series(imps, index=X.columns).sort_values(ascending=False)
rf_feat_imps.plot(kind='bar')
plt.xlabel('features')
plt.ylabel('importance')
plt.title('Best random forest classifier feature importances');

No significant difference in feature importances

Let's more onto oldformats 4 classes:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_df['oldformats4'],
                        test_size=0.25, random_state=42,
                        stratify=y_df['oldformats4'].values)
random_forest_classifier = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(random_forest_classifier, param_grid=grid_params, cv=2, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_, rf_grid.best_score_

In [ ]:
y_predrfold4_train = rf_grid.predict(X_train)
y_predrfold4_test = rf_grid.predict(X_test)
precision_recall_fscore_support(y_train, y_predrfold4_train, labels=['Not c', 'Low c', 'Mid c', 'High c']), precision_recall_fscore_support(y_test, y_predrfold4_test, labels=['Not c', 'Low c', 'Mid c', 'High c'])

Oldformats doesn't have good performance here, due to low c performing poorly and mid c not doing very well either.

In [ ]:
plt.subplots(figsize=(10, 5))
imps = rf_grid.best_estimator_.feature_importances_
rf_feat_imps = pd.Series(imps, index=X.columns).sort_values(ascending=False)
rf_feat_imps.plot(kind='bar')
plt.xlabel('features')
plt.ylabel('importance')
plt.title('Best random forest classifier feature importances');

Utterly typical

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_df['oldformats2'],
                        test_size=0.25, random_state=42,
                        stratify=y_df['oldformats2'].values)
random_forest_classifier = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(random_forest_classifier, param_grid=grid_params, cv=2, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_, rf_grid.best_score_

In [ ]:
y_predrfold2_train = rf_grid.predict(X_train)
y_predrfold2_test = rf_grid.predict(X_test)
precision_recall_fscore_support(y_train, y_predrfold2_train, labels=['No', 'Yes']), precision_recall_fscore_support(y_test, y_predrfold2_test, labels=['No', 'Yes'])

Again it does noticeably worse than tierchange formats, so I think we're likely to go with tierchange over oldformats due to the more balanced classes we created.

In [ ]:
plt.subplots(figsize=(10, 5))
imps = rf_grid.best_estimator_.feature_importances_
rf_feat_imps = pd.Series(imps, index=X.columns).sort_values(ascending=False)
rf_feat_imps.plot(kind='bar')
plt.xlabel('features')
plt.ylabel('importance')
plt.title('Best random forest classifier feature importances');

Typical and predictably weighted towards stats, though it's interesting that attacking stats are more important and defense is even lower than resistance index.

<a id="hierarchical_clustering"></a>
### Hierarchical Clustering

**Now, we are moving onto clustering.**  We want to try three kinds of clustering:
- Hierarchical
- KMeans
- Density based
- Meanshift

First of all, we can get a nice visualization of our hierarchical clustering via a dendrogram! We need to scale our data first, since we are using distance measures and there isn't a reason that features with larger or smaller distance scales should a priori be more important:

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
mergings = linkage(X_scaled, method='complete')
dendrogram(mergings,
           labels=X.index,
           leaf_rotation=90,
           leaf_font_size=4)
plt.show()

While beautiful and interesting, this dendrogram is far too busy to be of much use. Let's look more closely at the results of different cluster distance cutoff points:

In [ ]:
labels = fcluster(mergings, 10, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': X.index})
pairs['labels'].value_counts()

In [ ]:
labels = fcluster(mergings, 9, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': X.index})
pairs['labels'].value_counts()

A distance of 10 shows a good amount of clusters (16), not too many and not too few, but a lot of the clusters are really large so I wonder how much those can really mean. 9 still has a possibly tolerable amount at 28, but it's tending towards a lot so I don't think going down to 8 is a good idea.

Let's examine the clusters for a distance of 10 by examining the outliers, a few small and medium sized clusters, and a large cluster (and merge with the 'formats' feature so that we can see the target variable in each clustering):

In [ ]:
labels = fcluster(mergings, 10, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': X.index})
pairs['labels'].value_counts()

In [ ]:
pairs[pairs['labels'] == 15].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

In [ ]:
pairs[pairs['labels'] == 16].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

So our uniques are Mew and Shuckle, who both have very unique features. Shuckle has extremely high defense and special defense, and very low values for all of its other stats. Mew is the only pokemon that can learn ANY move in all of pokemon which is available to be learned (i.e. it doesn't include unique moves the level up sets of many pokemon, but it is BY FAR the largest learnset of any pokemon).

In [ ]:
pairs[pairs['labels'] == 1].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

Chansey and Blissey uniquely have by far the highest HP of all pokemon, and it's the reason that they are used as special walls due to the fact that they also have high special defense and a type (normal) that has few weaknesses (however their HP is really what stands out about them that the clustering is surely detecting).

In [ ]:
pairs[pairs['labels'] == 2].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

These are the other very high hp pokemon, just of a notably lower hp than Blissey and Chansey. Probably they are in different clusters because HP is the stats with the highest variance of all the stats in pokemon, so the other stats won't be able to split into as many small unique clusters like this. It also stands out that these pokemon aren't as competitive as Blissey and Chansey who both have quite high tiers, so that's probably another reason for the split (Blissey and Chansey are also more powerful). That said, I'm not sure it's ideal to split these high hp pokemon, so we'll see if they glom together in other forms of clustering.

In [ ]:
pairs[pairs['labels'] == 4].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

This one is harder to see the common thread in. There are fast and lightweight pokemon like Talonflame and Regieleki, and heavy bulky pokemon like Mudsdale, Stonjourner and Regidrago, all with quite different types and seemingly movesets as well, not to mention a large spread in terms of formats (although all of them are competitive, at least if they are a final evolution in their family). It's hard to see much common thread though, so I don't think this cluster is very good.

Let's start skipping by twos, since we don't need to look at every cluster, as we will ultimately take the clustering method and distance as hyperparamters in tuning our model.

In [ ]:
pairs[pairs['labels'] == 9].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

The common thread here is the ability "competitive" or something very similar to it (e.g. defiant), which must mean this ability is quite unique and important (in fact it is, it's hazard removal deterrent).

In [ ]:
pairs[pairs['labels'] == 6].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

These are pokemon who often have types like ghost or steel that have a lot of immunities and resistances, so they likely have a high resistance index.

In [ ]:
pairs[pairs['labels'] == 13].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

This is a lot of very powerful legendary or quasi-legendary pokemon, though Silvally seems like the weak link here, and there are a few non-legendaries like Marowak-Alola.

In [ ]:
pairs[pairs['labels'] == 10].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True).head(50)

This is just the first 50 from the largest indiscrimiate cluster, and this one indeed doesn't seem that informative, with great variety of types, stats, competitive viability, etc. But the fact that hierarchical clustering definitely detected some very sensible and reasonable clusters is promising, and even if only some of the clusters are reasonable in my final model, it may still increase how discerning the model is about unusual cases (since they'll belong to a cluster that will cause them to be treated differently, instead of a bigger standardized cluster like this).

That was all a clustering based on all of the features that have a chance of making it into my final model. It might be interesting to cluster by focusing on just part of the data, such as:
- stats
- abilities
- learnset
- (type information is already contained in merely one feature so there is no need to find clusters in one feature)

We will start with the stats. The values I found most useful during EDA_experiments_2 were between 5 and 3.5. 3.75 probably did the best job of breaking down the largest clusters with the highest value possible, but there were still arguably too many clusters with this value (34 total). We'll try a value of 5 (since it has the lowest number of clusters, 16, of the good values) to get an idea of what it will cluster together, but our ultimate value of choice will be a tunable hyperparameter.

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_df.loc[:, ['hp', 'atk', 'def', 'spa', 'spd', 'spe']])
mergings = linkage(X_scaled, method='complete')
labels = fcluster(mergings, 5, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': pokemon_df.index})
pairs['labels'].value_counts()

In [ ]:
pairs[pairs['labels'] == 6].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

In [ ]:
pairs[pairs['labels'] == 16].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

Shuckle was already unique in terms of stats, but now Kartana is seem as having very unique stats, and I agree because Kartana has grotesquely high physical stats and very low special stats and hp. It makes sense that Mew was displaced here since its learnset was so unique, not its stats.

In [ ]:
pairs[pairs['labels'] == 1].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

Chansey and Blissey, still taking on the role of the super high hp pokemon who are also powerfully competitive

In [ ]:
pairs[pairs['labels'] == 2].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

These are more high hp pokemon, but this time it's more inclusive since we specifically focused on stats, so that's even more useful

Let's start skipping by 2, since we don't need to dwell on every cluster, it just needs to be reasonable overall

In [ ]:
pairs[pairs['labels'] == 4].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

These are quite physically oriented (attack and defense) and also quite bulky (high hp and/or at least one of the defenses) pokemon, attack is usually higher than defense though

In [ ]:
pairs[pairs['labels'] == 3].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

These are also physically oriented pokemon but slanted notably more towards defense than attack, so like a mirror of the last one

In [ ]:
pairs[pairs['labels'] == 15].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True)

Weak baby pokemon, almost entirely uncompetitive

In [ ]:
pairs[pairs['labels'] == 14].merge(pokemon_data['formats'], how='inner', left_on='pokemon', right_index=True).head(50)

Also weak baby pokemon in the largest cluster (of which the above only shows the first 50).

Overall, I'd say that the clustering of stats performed more consistently and coherently than clustering based on all of the features of the pokemon, so I like it a lot.

Now let's try clustering by abilities:

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_abilities_df)
mergings = linkage(X_scaled, method='complete')
labels = fcluster(mergings, 10, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': pokemon_df.index})
pairs['labels'].value_counts()

330 clusters for 738 rows is absurdly unrealistic.  Let's try a larger distance.

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_abilities_df)
mergings = linkage(X_scaled, method='complete')
labels = fcluster(mergings, 30, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': pokemon_df.index})
pairs['labels'].value_counts()

59 clusters is still probably too many to be useful, especially when almost all of the pokemon are contained in one clusters and almost all the rest are unique. So ultimately, abilities is too sparse to be useful for clustering. We can try it for learnsets but I don't have high hopes:

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_learnsets_df)
mergings = linkage(X_scaled, method='complete')
labels = fcluster(mergings, 10, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': pokemon_df.index})
pairs['labels'].value_counts()

571 clusters for 738 is EVEN more absurdly unrealistic!

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_learnsets_df)
mergings = linkage(X_scaled, method='complete')
labels = fcluster(mergings, 50, criterion='distance')
pairs = pd.DataFrame({'labels': labels, 'pokemon': pokemon_df.index})
pairs['labels'].value_counts()

Even at a distance of 50, we still have the vast majority of pokemon in one cluster, and 37 clusters is still a rather large amount. This isn't as bad as abilities, which makes sense since pokemon share a lot more moves than abilities even though there are a lot more moves, but overall I can't see this being helpful, especially with most clusters being uniques or containing very few examples.

<a id="k_means"></a>
### k-means clustering

For our best clustering values in hierarchical clustering, we had somewhere between 15 and 30 clusters, so that's probably what we should aim for with kmeans since we need to specify it (and of course we will test that number as a hyperparameter)

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(X)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

These numbers look surprisingly balanced and reasonable. There aren't even any major outliers and the size increases very smoothly. We can look at a few smaller clusters, a few medium and a large one.

In [ ]:
pairs[pairs['labels'] == 13]

Now it looks like we have most of the high hp pokemon bunched together, INCLUDING Chansey and Blissey! This is promising.

In [ ]:
pairs[pairs['labels'] == 12]

Very high defense pokemon with rock, steel and ice typings

In [ ]:
pairs[pairs['labels'] == 10]

This is a stranger cluster and hard to see the coherence of, but it contains a lot of middle evolutions or second evolutions with semi-balanced stats, but there are a few exceptions like Talonflame, Crobat, Shiftry, Poliwhirl etc. (the former two of whom are surprisingly fast)

In [ ]:
pairs[pairs['labels'] == 8]

These are a lot of faster physical attackers

In [ ]:
pairs[pairs['labels'] == 6]

A set of uncompetitive baby pokemon.

So overall, this clustering is pretty good. Its results are comparable to hierarchical clustering, but I greatly prefer the relative sizes of the clusters, so k means will probably end up being the preferred option. This may make sense since it is a more artificial clustering method, and we didn't have good natural clusters in the data as we saw with our PCA visualization, so forcing more artificial clusters may be more suited to our data.

Now let's try the same thing for just the stats:

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(pokemon_df.loc[:, ['hp', 'atk', 'def', 'spa', 'spd', 'spe']])
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Still very good numbers overall, with just one notable outlier:

In [ ]:
pairs[pairs['labels'] == 18]

Of course, Chansey and Blissey are unique in having a very high hp cluster again

In [ ]:
pairs[pairs['labels'] == 17]

We have our very high defense pokemon in this cluster, it seems

In [ ]:
pairs[pairs['labels'] == 1]

These are very physically oriented pokemon, but with middling to higher speed

In [ ]:
pairs[pairs['labels'] == 9]

Also quite physical oriented pokemon, but who have lower speed (reflected in seemingly lower average rating in the target variable as compared to the higher speed physical cluster we looked at)

In [ ]:
pairs[pairs['labels'] == 15]

Often quite fast pokemon with good physical attack

I think we're getting the point here, and like using stats with hierarchical clustering, these k means clusters with only stats are very coherent and useful.

Let's see if k means works any better for abilities due to its relative artificiality:

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(pokemon_abilities_df)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

This is actually much better than with hierarchical clustering. There is still one huge cluster, but lots of smaller one's that might be meaningful and we should investigate those this time.

In [ ]:
pairs[pairs['labels'] == 9]

This group has Telepathy and Synchronize in common, makes sense

In [ ]:
pairs[pairs['labels'] == 8]

These all have Flash Fire and Flame Body in common, also makes sense, so it looks like these clusters will be tight groupings of abilities

In [ ]:
pairs[pairs['labels'] == 15]

This seems to be the Intimidate group

In [ ]:
pairs[pairs['labels'] == 17]

This is the Regenerator group

In [ ]:
pairs[pairs['labels'] == 14]

Seems to be a combination of Inner Focus, Pressure, and Unnerve

And we can imagine that the larger cluster will contain many less distinct abilities.  I do think this abilities clustering could be useful for k means since some of these abilities might give a lot of information, and the large cluster will probably just be treated more or less normally.

Let's try learnsets:

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(pokemon_learnsets_df)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

This is actually almost shockingly useful, compared to the hierarchical clustering version.

In [ ]:
pairs[pairs['labels'] == 17]

Some dual typed water pokemon, makes sense

In [ ]:
pairs[pairs['labels'] == 1]

A lot of dual psychic types with rock or steel or dual rock types with steel or ice, makes sense.

These aren't as interesting because we can't directly compare the learnsets since they are too vast in most cases, but this seems surprisingly promising. That means for kmeans clustering, we could try doing it with just the features going into the model, or we can try doing it in pieces with stats, abilities and learnsets (3 separate clusterings).

<a id="DBSCAN"></a>
### DBSCAN: Density-Based Clustering

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusts = dbscan.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

So it basically detected 16 pokemon as outliers:

In [ ]:
pairs[pairs['labels'] == 0]

Naturally, Silvally is very similar to itself and thus has very high density! Makes sense. But overall, this clustering wasn't useful at all if most pokemon are in a single cluster (-1, indicating they are all noise that isn't located in a section with notable density). We can try some different eps values though, as it is the main parameter. min_samples as 5 is appropriate since groups smaller than that aren't very meaningful for our analysis.

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
dbscan = DBSCAN(eps=2.5, min_samples=5)
clusts = dbscan.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

This is still mostly noise.

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
dbscan = DBSCAN(eps=4.5, min_samples=5)
clusts = dbscan.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Even here, although not many are noise (cluster -1), most pokemon still end up getting thrown into a single dense cluster. That of course isn't useful at all.

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
dbscan = DBSCAN(eps=7.5, min_samples=5)
clusts = dbscan.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Now it's just all a single cluster. I think this algorithm likely isn't useful for our purposes. We can try it with stats:

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_df.loc[:, ['hp', 'atk', 'def', 'spa', 'spd', 'spe']])
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusts = dbscan.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Mostly noise

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_df.loc[:, ['hp', 'atk', 'def', 'spa', 'spd', 'spe']])
dbscan = DBSCAN(eps=2.5, min_samples=5)
clusts = dbscan.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

And another single giant cluster. I think we're not going to use DBSCAN in our model.

<a id="MeanShift"></a>
### Mean shift clustering

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
meanshift = MeanShift()
clusts = meanshift.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

So far it seems like a similar story to DBSCAN. We can try without scaling:

In [ ]:
meanshift = MeanShift()
clusts = meanshift.fit_predict(X)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Basically the same. Let's try with stats:

In [ ]:
X_scaled = StandardScaler().fit_transform(pokemon_df.loc[:, ['hp', 'atk', 'def', 'spa', 'spd', 'spe']])
meanshift = MeanShift()
clusts = meanshift.fit_predict(X_scaled)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Basically the same. Let's try abilities and learnsets:

In [ ]:
meanshift = MeanShift()
clusts = meanshift.fit_predict(pokemon_abilities_df)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

In [ ]:
meanshift = MeanShift()
clusts = meanshift.fit_predict(pokemon_learnsets_df)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Predictably, it wasn't useful

<a id="select_k_best"></a>
### SelectKBest

This is another fairly easy method that we can use to get feedback on which of our features are the best for our model and which of our features might not be useful. I wouldn't use this method alone since the ANOVA F-value between the label and feature might not be the best method to decide, but it is useful as feedback and it will be especially good evidence if the answers SelectKBest gives (as to whether features are useful or not) are similar to what is given by other methods.

In [ ]:
selectbest = SelectKBest(k='all')
selectbest.fit(X, y_df['formats'])
bestscores = pd.Series(selectbest.scores_, index=X.columns)
bestscores.sort_values(ascending=False)

In [ ]:
plt.plot(bestscores.sort_values(ascending=False))
plt.xticks(rotation = 90)
plt.show()

I notice a few things here: Unique Powerful Move is rather higher than usual, and basically all features than stats, moves, resistance index and unique powerful move are very low. This does confirm what we thought about the most useful features, but it doesn't tell us much about the least useful features since most of the features have pretty low values in common.

<a id="tsne"></a>
### T-distributed Stochastic Neighbor Embedding (TSNE)

Let's do T-SNE next, because it's a great way to visualize high dimensional data. We can try it with the general X, but I suspect it won't find too much there. I'm more hopeful of it finding something in stats though, and maybe that could give us a clue as to a useful number of clusters.

A seemingly good idea would be coloring the plot based on clusters! So I can reload my clustering from before, starting with the kmeans clustering for X.

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(X)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

Perplexity is an important hyperparamter of this algorithm.  I think we'll try 2, 5, 30, 50 and 100 like they do in this distill article: https://distill.pub/2016/misread-tsne/, using a for loop (I may have to go a bit higher for the amount of points I have, so I might throw 250 and 500 in there)

In [ ]:
perplexity = [2, 5, 30, 50, 100, 500]

fig, axes = plt.subplots(6, figsize=(12, 12*6))

for p, axis in zip(perplexity, axes):
    tsne = TSNE(n_components=2, perplexity=p, n_iter=1000, random_state=42, method='exact')
    embedded = pd.DataFrame(tsne.fit_transform(X), index=pokemon_data.index)
    embedded['labels'] = pairs['labels'].astype('category')
    sns.scatterplot(ax=axis, x=embedded[0], y=embedded[1], data=embedded, hue=embedded['labels'], palette=sns.color_palette(n_colors=20))

plt.show()

We can see the TSNE plots capturing the 20 KMeans clusters very well, especially in the first 4, so that is a very good sign about the validity and usefulness of our clusters. It also seems possible that a different number of clusters could be more ideal though, so it still makes sense to take the number of clusters as a hyperparameter.

Let's try the same thing with stats:

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(pokemon_df.loc[:, ['hp', 'atk', 'def', 'spa', 'spd', 'spe']])
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

In [ ]:
perplexity = [2, 5, 30, 50, 100, 500]

fig, axes = plt.subplots(6, figsize=(12, 12*6))

for p, axis in zip(perplexity, axes):
    tsne = TSNE(n_components=2, perplexity=p, n_iter=1000, random_state=42, method='exact')
    embedded = pd.DataFrame(tsne.fit_transform(pokemon_df.loc[:, ['hp', 'atk', 'def', 'spa', 'spd', 'spe']]), index=pokemon_data.index)
    embedded['labels'] = pairs['labels'].astype('category')
    sns.scatterplot(ax=axis, x=embedded[0], y=embedded[1], data=embedded, hue=embedded['labels'], palette=sns.color_palette(n_colors=20))

plt.show()

It's very similar results to X, which is good. The last plot is pretty chaotic but they all seem to capture a basic structure similar to the clusters.  Now let's do abilities and learnsets:

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(pokemon_abilities_df)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

In [ ]:
perplexity = [2, 5, 30, 50, 100, 500]

fig, axes = plt.subplots(6, figsize=(12, 12*6))

for p, axis in zip(perplexity, axes):
    tsne = TSNE(n_components=2, perplexity=p, n_iter=1000, random_state=42, method='exact')
    embedded = pd.DataFrame(tsne.fit_transform(pokemon_abilities_df), index=pokemon_data.index)
    embedded['labels'] = pairs['labels'].astype('category')
    sns.scatterplot(ax=axis, x=embedded[0], y=embedded[1], data=embedded, hue=embedded['labels'], palette=sns.color_palette(n_colors=20))

plt.show()

The abilities clusters might even be the most interesting so far, since there is a lot of blue scattered about from the very large and indistinct clusters, but there are these tightly clustered balls of various clusters within that representing smaller clusters, especially in the first 4 or 5 plots. I definitely think something like this could be useful information for the model.

Let's try learnsets:

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
clusts = kmeans.fit_predict(pokemon_learnsets_df)
pairs = pd.DataFrame({'labels': clusts, 'pokemon': pokemon_df.index, 'formats': pokemon_data['formats']})
pairs['labels'].value_counts()

In [ ]:
perplexity = [2, 5, 30, 50, 100, 500]

fig, axes = plt.subplots(6, figsize=(12, 12*6))

for p, axis in zip(perplexity, axes):
    tsne = TSNE(n_components=2, perplexity=p, n_iter=1000, random_state=42, method='exact')
    embedded = pd.DataFrame(tsne.fit_transform(pokemon_learnsets_df), index=pokemon_data.index)
    embedded['labels'] = pairs['labels'].astype('category')
    sns.scatterplot(ax=axis, x=embedded[0], y=embedded[1], data=embedded, hue=embedded['labels'], palette=sns.color_palette(n_colors=20))

plt.show()

The structure of this one is probably the strangest, because in the lower perplexity plots, you get two things: some very widely distributed clusters (which isn't a good sign) even though there aren't any particularly large clusters like there were for abilities, and also some tightly clustered balls that contain three or more clusters, which makes little sense.

However, the very high 500 perplexity plot seems to separate clusters best and most consistently, even though there aren't tightly clustered balls anymore (which may make sense since the learnsets are a lot more complex, and thus perhaps at far greater distance, than the stats and abilities and overall features going into my model). Probably the learnsets have the lowest quality so far though, but not low enough to convince me not to try clustering with them to see how it affects model performance.

<a id="heatmap"></a>
### Heatmap of Feature Correlations

Now we want to look at a heatmap of our features to see what mutual correlations they have (at least looking for those that are notable since most obviously won't be). This can potentially tell us whether any of our features might be redundant for the model (so that we can either not use them or fold them into more useful features).

In [ ]:
plt.subplots(figsize=(12,10))
sns.heatmap(X.corr());

What stands out as being of most interest here?
- All of the stats have decently high mutual correlation. But we certainly need all the stats, since they consistently perform the best out of all of our features, so we won't be getting rid of anything there
- Speed has a decent correlation with being a pivot (having pivot moves), but those two things don't seem similar enough that I could get rid of one or the other over it, especially since one of them is a stat which is one of our basic building blocks
- Attack has high correlation with physical cutoff and coverage, and likewise special attack has high correlation with special cutoff and coverage, but I can't get rid of one or the other since those are our best features (stats and move cutoffs). This correlation just seems very natural
- HP Drain and HP recovery have a somewhat decent correlation, so it might be worth considering folding them into the same feature since they have a similar role of restoring the hp of a pokemon (it's just that some do damage whereas some restore directly, but we still have cindex as a standardized means of comparing the separate hierarchies of those features to combine them into a single hierarchy)
- Misc status also has a decent correlation with special cutoff and coverage, but those are so different that it's hard to imagine combining them and since they are different but important kinds of moves, removing both of them doesn't seem realistic. Misc status might also become more predictive if I fold some of the other less useful features into it
- Probably the most important insight: physical cutoff has very high correlation with physical coverage, just as special cutoff has with special coverage, just as ability cutoff has with best ability. Those are probably the strongest candidate for features to remove; I may not all the features for physical and special attack, and especially not for abilities. For abilities we will very likely use Best Ability over Ability Cutoff 3 (simply removing the latter), and for the physical and special moves, I may just treat using one or both of them as a hyperparameter and check model performance

<a id="target_plots"></a>
### Plotting Features against the Target

Now we can try doing scatterplots of each feature against the target feature to see if anything noteworthy pops out that indicates something we might want to pay attention to:

In [ ]:
X_withformats = X.copy()
X_withformats['format codes'] = y_df['format codes']

In [ ]:
def scatterplots(columns, ncol=None, figsize=(15, 8)):
    if ncol is None:
        ncol = len(columns)
    nrow = int(np.ceil(len(columns) / ncol))
    fig, axes = plt.subplots(nrow, ncol, figsize=figsize, squeeze=False)
    fig.subplots_adjust(wspace=0.5, hspace=0.6)
    for i, col in enumerate(columns):
        ax = axes.flatten()[i]
        ax.scatter(x = col, y = 'format codes', data=X_withformats, alpha=0.5)
        ax.set(xlabel=col, ylabel='competitiveness')
    nsubplots = nrow * ncol    
    for empty in range(i+1, nsubplots):
        axes.flatten()[empty].set_visible(False)

In [ ]:
scatterplots(X.columns, ncol=4, figsize=(15, 15))

It's good that we looked at this, since some of these features don't seem to be doing what we'd like them to. For our best features, like stats, you can see their average largely increasing with their competitiveness.

- We have things like Entry Hazards and Hazard Removal.  These may be important aspects of the game, but increases in their values don't seem to correlate that well with competitiveness, with the highest value having the worst target variable results in some cases! Even removal deterrent isn't doing great at its best value
- Pivot has high correlation with speed and the 2 value isn't very good, so maybe it's best to throw into the better move cutoffs like physical, special and misc status in some cases
- Setup actually seems like a good stand alone feature because you can see higher values correlating to competitiveness whereas low values of this are never competitive
- Priority is not doing so well, with very little differentiation between tiers and the highest value of it seeming the lightest when it comes to high competitive rankings
- HP Drain and Recovery really don't seem very good either, and I was already considering folding them into a combined feature together so I may do exactly that
- Weather Set is really unique, but maybe I should just consider it a good ability?  As a standalone it doesn't seem super predictive.  Weather Gimmick seems even worse in this regard, with higher values being rather uncompetitive in some cases. It's possible that combining them together could also help
- Misc status isn't even great, but it's not terrible and if we fold more into it, maybe that highest value will populated more. Also, we need to aggregate information about status moves somewhere so I don't think I can justify getting rid of it

<a id="what_test"></a>
### What To Test During Pre-Processing and Modeling

- two-stage model or one-stage model
- 4 class vs 7 class model
- how to split a 4 class model: like we did in tierchange_df formats, in oldformats (which was ZU/PU+NU+RU/UU+OU/Ubers), or even using a new split for tierchange_df formats to maximize both class sizes and distinctiveness of Ubers (ZU/PU/NU+RU+UU+OU/Ubers)
- gridsearchcv for some hyperparameters for each model
- whether to use clustering or not
- how many kmeans clusters (we're just going to do it using the overall features for now, and stats/abilities/learnsets separately, we aren't going to do combinations)

to do with features:
- remove: ability cutoff, unique powerful ability
- fold weather set into weather gimmick
- fold hp drain and hp recovery together into a recovery feature
- considering: removal deterrent (could arguably just remove since its abilities), hazard removal, cleric, entry hazards (all 3 of those might go into misc status)

In [ ]:
pokemon_data.to_csv('./data/pokemon_data.csv')